In [ ]:
import os
import sys
print("Python version" + sys.version)
os.getcwd()
print(sys.executable)

In [ ]:
import numpy as np
np.random.seed(123)
import pandas as pd
import scipy
import itertools

import umap
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import scanpy as sc
import anndata as ad
import scvelo as scv
from tqdm.notebook import tqdm

from pathlib import Path

TF_ENABLE_ONEDNN_OPTS=0

In [ ]:
import venn

In [ ]:
import cellrank as cr

In [ ]:
cr.__version__

In [ ]:
from cellrank.kernels import CytoTRACEKernel

In [ ]:
import scanpy.external as sce

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score, homogeneity_completeness_v_measure, adjusted_mutual_info_score
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
sc.settings.verbosity = 1
sc.logging.print_header()
sc.settings.set_figure_params(dpi=300, facecolor='white')

In [ ]:
# remove weird grid from scvelo
plt.rcParams['axes.grid'] = False

plt.rcParams["savefig.format"] = 'pdf'
plt.rcParams["savefig.dpi"] = '100'

import matplotlib as mpl
mpl.use('pgf')

from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.backend_bases import register_backend
register_backend('pdf', FigureCanvasPgf)

In [ ]:
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg', dpi = 300)

In [ ]:
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = '8'

In [ ]:
# revised from Stefan's cell type signature
signatures_path_ = '../cell_type_from_stefan/scrnaseq_signature_collection/'
from score_and_classify import *

In [ ]:
# for using R plotting
%load_ext rpy2.ipython
#%Rdevice svg

def plot_umap_legend_only(fig, size=8):
    ax=fig.axes[0]
    label_params = ax.get_legend_handles_labels() 

    figl, axl = plt.subplots()
    axl.axis(False)
    axl.legend(*label_params, loc="center", bbox_to_anchor=(0.5, 0.5), prop={"size":size}, frameon= False)
;

In [ ]:
%%R

# R setting
suppressMessages(library('tidyverse'))
suppressMessages(library('grid'))
suppressMessages(library('ComplexUpset'))
suppressMessages(library('ggvenn'))

plot_theme_font_size = function(fontsize = 8){
    plot_theme =  theme(title = element_text(size = fontsize, family = "dajavu sans", colour = 'black', hjust = 0.5),
                   axis.title = element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   axis.text = element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   legend.title =  element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   legend.text=element_text(size=fontsize, family = "dajavu sans", colour = 'black'),
                   text = element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   strip.text = element_text(size = fontsize, family = "dajavu sans", colour = 'black'),
                   plot.margin = margin(0,0,0,0,"cm")) 
    plot_theme
}

In [ ]:
new_data_folder = '/fast/users/twei_m/work/crc/datasets_new_preprocessing'

In [ ]:
adata_all = sc.read(Path(new_data_folder)/'202307_adata_all_full_cell_type_annotation.h5')

In [ ]:
adata_epi = sc.read(Path(new_data_folder)/'202310_CB_epi_Numbat_Scitcem_inferCNV_icms_Uhlitz_scanvi_resolved_identity.h5')

In [ ]:
adata_epi_noTC_cleanCI.write(Path(new_data_folder)/'202401_adata_epi_consensus_call_no_TC.h5')


In [ ]:
adata_all_noTC_cleanCI.write(Path(new_data_folder)/'202401_adata_all_consensus_call_no_TC.h5')


In [ ]:
adata_epi.write(Path(new_data_folder)/'202401_adata_epi_consensus_call_with_TC.h5')


In [ ]:
adata_all.write(Path(new_data_folder)/'202401_adata_all_consensus_call_with_TC.h5')


In [ ]:
adata_epi.uns['log1p']['base'] = None

In [ ]:
tsamples = ['p007t', 'p008t', 'p009t1','p009t2', 'p013t', 'p014t', 'p016t', 
           'p020t', 'p021t', 'p026t', 'p035t'] 

In [ ]:
samples = ['p007n', 'p008n', 'p009n1', 'p009n2','p013n', 'p014n', 'p016n', 
           'p020n', 'p021n'] + tsamples

In [ ]:
adata_epi.uns['tumour_normal_normal_colors'] = ['#ff7f0e', '#9b1ee3','#1f77b4','#d3d3d3']

In [ ]:
cell_type_list = adata_epi.obs['Uhlitz_scANVI'].cat.categories.values

In [ ]:
cell_type_list

In [ ]:
# fix TCs
# genomically tumour = Tumour like
adata_epi.obs['Uhlitz_scANVI_noTC'] = None

for i in np.arange(0, adata_epi.shape[0]):
    if adata_epi.obs['cell_identity'][i] == 'genomically_tumour':
        adata_epi.obs['Uhlitz_scANVI_noTC'][i] = 'Tumour-like'        
    elif 'TC' in adata_epi.obs['Uhlitz_scANVI'][i]:
        adata_epi.obs['Uhlitz_scANVI_noTC'][i] = 'Tumour-like (normal_sample)'        
    else: 
        adata_epi.obs['Uhlitz_scANVI_noTC'][i] = adata_epi.obs['Uhlitz_scANVI'][i]
        
adata_epi.obs['Uhlitz_scANVI_noTC'] = adata_epi.obs['Uhlitz_scANVI_noTC'].astype('category')

In [ ]:
adata_epi.obs['Uhlitz_scANVI_noTC'].cat.categories

In [ ]:
adata_epi.obs['Uhlitz_scANVI_noTC'] = adata_epi.obs['Uhlitz_scANVI_noTC'].cat.reorder_categories(['Tumour-like', 
                                                                                                  'Tumour-like (normal_sample)',
                                                                                                  'Stem', 'Stem/TA', 
                                                           'Tuft',
                                                           'Immature Goblet', 'Goblet',
                                                           'Enterocyte progenitor', 'Enterocytes'])

In [ ]:
Uhlitz_scANVI_noTC_color_list = ['#AA0000', '#808080',
                                              '#00CCFF', '#0072b1',
                                              '#FF9900',
                                              '#990099', '#660099',
                                              '#999900', '#336600']

adata_epi.uns['Uhlitz_scANVI_noTC_colors'] = Uhlitz_scANVI_noTC_color_list

In [ ]:
epi_color_list = ['#999900', '#336600', '#660099', '#990099', '#00CCFF', '#0072b1',
                  '#AA0000', '#FF0000', '#FF0099', '#EB9999', '#FF9900']

In [ ]:
Uhlitz_scANVI_noTC_color_list_after = ['#00CCFF', '#0072b1',
                                              '#FF9900',
                                              '#990099', '#660099',
                                              '#999900', '#336600']
# no Tumour-like

In [ ]:
Uhlitz_scANVI_noTC_color_list = ['#AA0000', 
                                              '#00CCFF', '#0072b1',
                                              '#FF9900',
                                              '#990099', '#660099',
                                              '#999900', '#336600']

In [ ]:
# fix sample column: p009t and P to be capital
# genomically tumour = Tumour like
adata_epi.obs['sample_cap'] = None

for i in np.arange(0, adata_epi.shape[0]):
    if adata_epi.obs['sample'][i] in ['p009t1', 'p009t2']:
        adata_epi.obs['sample_cap'][i] = 'P09t'        
    elif adata_epi.obs['sample'][i] in ['p009n1', 'p009n2']:
        adata_epi.obs['sample_cap'][i] = 'P09n'
    else: 
        adata_epi.obs['sample_cap'][i] = adata_epi.obs['sample'][i].replace('p0', 'P')
        
adata_epi.obs['sample_cap'] = adata_epi.obs['sample_cap'].astype('category')

# patient to cap
adata_epi.obs['patient'] = adata_epi.obs['patient'].astype('str').apply(lambda x: 
                                                                        x.replace('p0', 'P')).astype('category')

In [ ]:
adata_epi.obs['sample_cap'].cat.categories

In [ ]:
adata_epi.obs['patient'].cat.categories

In [ ]:
# ct pseudotime
# format hack
adata_epi.layers["spliced"] = adata_epi.X
adata_epi.layers["unspliced"] = adata_epi.X

# calculate 
scv.pp.moments(adata_epi, n_pcs=30, n_neighbors=30)
ctk = CytoTRACEKernel(adata_epi).compute_cytotrace()
ctk.compute_transition_matrix(threshold_scheme="soft", nu=0.5)


In [ ]:
adata_epi_noTC_cleanCI = adata_epi[(adata_epi.obs['Uhlitz_scANVI_noTC'] != 'Tumour-like (normal_sample)') & #37670
                                   (adata_epi.obs['cell_identity'] != 'no confident assignment')] # 37316


In [ ]:
sample_color_map = dict(zip(adata_epi_noTC_cleanCI.obs['Uhlitz_scANVI_noTC'].cat.categories, 
                            adata_epi_noTC_cleanCI.uns['Uhlitz_scANVI_noTC_colors']))

In [ ]:
sample_color_map

In [ ]:
removed_TC_id = list(set(adata_epi.obs.index) - set(adata_epi_noTC_cleanCI.obs.index))

In [ ]:
len(removed_TC_id)

In [ ]:
# for all cells
# fix sample column: p009t and P to be capital
# genomically tumour = Tumour like
adata_all.obs['sample_cap'] = None

for i in np.arange(0, adata_all.shape[0]):
    if adata_all.obs['sample'][i] in ['p009t1', 'p009t2']:
        adata_all.obs['sample_cap'][i] = 'P09t'        
    elif adata_all.obs['sample'][i] in ['p009n1', 'p009n2']:
        adata_all.obs['sample_cap'][i] = 'P09n'
    else: 
        adata_all.obs['sample_cap'][i] = adata_all.obs['sample'][i].replace('p0', 'P')
        
adata_all.obs['sample_cap'] = adata_all.obs['sample_cap'].astype('category')

# patient to cap
adata_all.obs['patient'] = adata_all.obs['patient'].astype('str').apply(lambda x: 
                                                                        x.replace('p0', 'P')).astype('category')

In [ ]:
adata_all.obs['sample_cap'].cat.categories

In [ ]:
adata_all.obs['patient'].cat.categories

In [ ]:
adata_all_noTC_cleanCI = adata_all[~adata_all.obs.index.isin(removed_TC_id)]

### Numbers needed in Result

In [ ]:
# 73294 cells in total
adata_all.shape

In [ ]:
# 30184 cells from normal samples, 43110 cells from tumour samples
adata_all.obs['sample_origin'].value_counts()

In [ ]:
# 39168 epithelial cells, 31663 immune cells, and 2463 stromal cells
adata_all.obs['celltype_1a'].value_counts()

In [ ]:
# within epithelial cells, 21545 cells from normal samples, 17623 cells from tumour sample
adata_epi.obs['sample_origin'].value_counts()

In [ ]:
# agreement between Numbat and Scitcem
adata_epi.obs['tumour_normal_normal'].value_counts()

In [ ]:
# cell identity
adata_epi.obs['cell_identity'].value_counts()

In [ ]:
# cell identities not agreed by all four tools are 4666
17623-7320-5637

### Figure 1abcdef
- a. UMAP of all the patients and all the cells (epi, imm, str), coloured by patient
- b. UMAP of all the patients and all the cells (epi, imm, str), coloured by tissue origin
- c. UMAP of all the patients and all the cells (epi, imm, str), coloured by cell type compartment
- d. UMAP of epithelial cells and coloured by tissue origin
- e. UMAP of epithelial cells and coloured by iCMS prediction of cells from tumour samples
- f. UMAP of epithelial cells and coloured by inferCNV result

In [ ]:
fig = sc.pl.umap(adata_all, color=['patient'], 
              legend_loc='right margin', size = 1, frameon = False,return_fig=True)

In [ ]:
# Figure 1b
# UMAP of all the patients and all the cells (epi, imm, str), coloured by tissue origin

adata_all.uns['sample_origin_colors'] = adata_epi.uns['sample_origin_colors']

fig = sc.pl.umap(adata_all, color=['sample_origin'], 
               ncols=1, legend_loc='right margin', size = 1, frameon = False,return_fig=True)

In [ ]:
adata_all.obs['celltype_1a'].cat.rename_categories({'epi':'Epithelial cell',
                                                   'imm':'Immune cell',
                                                   'str':'Stromal cell'}, inplace = True)

In [ ]:
adata_all.uns['celltype_1a_colors'] = ['#144FAC', '#ED5D47', '#4EAC57']


In [ ]:
# Figure 1c
# UMAP of all the patients and all the cells (epi, imm, str), coloured by cell type compartment
fig = sc.pl.umap(adata_all,  color=['celltype_1a'], 
               ncols=1, legend_loc='right margin', size = 1,
               title = '', frameon = False, return_fig=True)


In [ ]:
# Figure 1d
# UMAP of epithelial cells and coloured by tissue origin
fig = sc.pl.umap(adata_epi, color=['normal_vs_MS'], 
               ncols=1, legend_loc='right margin', size = 2,
              title = '', frameon = False, return_fig=True)


In [ ]:
fig = sc.pl.umap(adata_epi, color=['patient'], 
               ncols=1, legend_loc='right margin', size = 2,
              title = '', frameon = False, return_fig=True)


In [ ]:
# Figure 1e 
# UMAP of epithelial cells and coloured by iCMS prediction of cells from tumour samples
fig = sc.pl.umap(adata_epi, color=['iCMS_scANVI'], 
               ncols=1, legend_loc='right margin', size = 2,
              title = '', frameon = False, return_fig=True)

In [ ]:
adata_epi.obs['inferCNV_result'] = adata_epi.obs['inferCNV_result'].cat.rename_categories({
    'failed_sample':'NA'})


In [ ]:
# Figure 1f 
# UMAP of epithelial cells and coloured by inferCNV result

fig = sc.pl.umap(adata_epi, color=['inferCNV_result'], 
               ncols=1, legend_loc='right margin', size = 2,
              title = '', frameon = False, return_fig=True)

In [ ]:
iCMS_and_inferCNV = adata_epi.obs[['inferCNV_result', 'iCMS_scANVI', 
                                   'iCMS_scANVI_pred_only','sample_cap', 'sample_origin', 'Uhlitz_scANVI']]

In [ ]:
iCMS_and_inferCNV[(iCMS_and_inferCNV['iCMS_scANVI_pred_only'] != 'Normal') & 
                  (iCMS_and_inferCNV['sample_origin'] == '\nnormal\nsample\n')]['Uhlitz_scANVI'].value_counts().sort_index()

In [ ]:
%%R -i iCMS_and_inferCNV -w 700 -h 500 -r 200

# bar plot of iCMS result per tumour sample

df = iCMS_and_inferCNV %>%
filter(sample_origin != '\nnormal\nsample\n') %>%
group_by(sample_cap, iCMS_scANVI) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct(sample_cap, iCMS_scANVI, sum, sum_sample) %>%
arrange(sample_cap)


p = df %>%
ggplot(aes(x = sum, y = sample_cap, fill = iCMS_scANVI)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#FFB84C', 
                             '#E96479',
                             '#1f77b4'
                            )) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90, family = "dajavu sans")) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,labels = paste0(rev(unique(df$sample_cap)), ' (', rev(unique(df$sum_sample)), ')'))

print(p)

#grid.newpage()
#grid.draw(cowplot::get_legend(p))


ggsave(file="iCMS_bar.svg", width = 8, height = 5, units = "cm", dpi = 300)

In [ ]:
%%R -i iCMS_and_inferCNV -w 900 -h 500 -r 200

# Sup. Fig
# bar plot of iCMS prediction in normal sample

df = iCMS_and_inferCNV %>%
filter(sample_origin == '\nnormal\nsample\n') %>%
group_by(sample_cap, iCMS_scANVI_pred_only) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct(sample_cap, iCMS_scANVI_pred_only, sum, sum_sample, Uhlitz_scANVI) %>%
arrange(sample_cap)


g = df %>%
ggplot(aes(x = sum, y = sample_cap, fill = iCMS_scANVI_pred_only)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#FFB84C', 
                             '#E96479',
                             '#1f77b4'
                            )) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_y_discrete(limits = rev,labels = paste0(rev(unique(df$sample_cap)), ' (', rev(unique(df$sum_sample)), ')'))

print(g)


# the not normal cell
df2 = iCMS_and_inferCNV %>%
filter(sample_origin == '\nnormal\nsample\n' & iCMS_scANVI_pred_only != 'Normal') %>%
group_by(sample_cap, Uhlitz_scANVI) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct(sample_cap, sum, sum_sample, Uhlitz_scANVI) %>%
arrange(sample_cap)

df2 %>%
ggplot(aes(x = sum, y = sample_cap, fill = Uhlitz_scANVI)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#999900', '#336600', '#660099', '#990099', '#00CCFF', '#0072b1',
                             '#AA0000', '#FF0000', '#FF0099', '#EB9999', '#FF9900'),
                  name = 'Epithelial cell type') +
theme_void() + 
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_y_discrete(limits = rev)  +
guides(fill=guide_legend(ncol=2))



In [ ]:
%%R -i iCMS_and_inferCNV -w 700 -h 500 -r 200

# bar plot of inferCNV result per sample

df = iCMS_and_inferCNV %>%
filter(sample_origin != '\nnormal\nsample\n') %>%
group_by(sample_cap, inferCNV_result) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct(sample_cap, inferCNV_result, sum, sum_sample) %>%
arrange(sample_cap)


p = df %>%
ggplot(aes(x = sum, y = sample_cap, fill = inferCNV_result)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#ff7f0e', 
                             '#1f77b4',
                             '#9b1ee3'
                            )) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,labels = paste0(rev(unique(df$sample_cap)), ' (', rev(unique(df$sum_sample)), ')'))


print(p)

ggsave(file="inferCNV_bar.svg", width = 8, height = 5, units = "cm", dpi = 300)

### Figure 2abcdef
#### Benedikt's part
- a. Scitcem scheme
- b. number of SNVs
- c. benchmark result: sensitivity, efficiency
- d. higher tumour content & sensitivity and number of SNV
- e. model params vs tumour content (missing)
- subsampling of CRC data mean read /cell
- TPR across #SNV, fraction cells without SNVs, mean # expressed SNVs, mean # reads
#### Cindy's part
- f UMAP of Scitcem calls

In [ ]:
# Figure 2f
# UMAP of Scitcem result
fig = sc.pl.umap(adata_epi, color=['scitcem_call'], 
               ncols=1, legend_loc='right margin', size = 2,
              title = '', frameon = False, return_fig=True)

### Figure 3abcd
- a. UMAP of Numbat result
- b. Upset plot of all the cell identity assignment
- c. Upset plot of Numbat and Scitcem
- d. bar plot of resolved cell identities in each tumour samples (both agree + more in Methods)

In [ ]:
# Figure 3a
# UMAP of Numbat result
fig = sc.pl.umap(adata_epi, color=['numbat'], 
               ncols=1, legend_loc='right margin', size = 2,
              title = '', frameon = False, return_fig=True)

In [ ]:
# cell identities not agreed by all four tools:
17623-7320-5637

In [ ]:
# 1296 cell inferCNV normal
inferCNV_only_normal_cells = adata_epi[(adata_epi.obs['sample_origin'] == 'tumour\nsample') &
         (adata_epi.obs['scitcem_call'] == 'tumour\n(tumour sample)') &
         (adata_epi.obs['numbat'] == 'tumour\n(tumour sample)') &
         (adata_epi.obs['inferCNV_result'] != 'CNA\n(tumour sample)') &
         (adata_epi.obs['iCMS_scANVI'] != 'normal\n(tumour sample)')]

In [ ]:
inferCNV_only_normal_cells.obs['sample_cap'].value_counts().sort_index()

In [ ]:
inferCNV_only_normal_cells.obs[['sample_cap','clone_opt']].value_counts().sort_index()

In [ ]:
# 1214 cell Scitcem normal
scitcem_only_normal_cells = adata_epi[(adata_epi.obs['sample_origin'] == 'tumour\nsample') &
         (adata_epi.obs['scitcem_call'] != 'tumour\n(tumour sample)') &
         (adata_epi.obs['numbat'] == 'tumour\n(tumour sample)') &
         (adata_epi.obs['inferCNV_result'] == 'CNA\n(tumour sample)') &
         (adata_epi.obs['iCMS_scANVI'] != 'normal\n(tumour sample)')]

In [ ]:
scitcem_only_normal_cells.obs['sample_cap'].value_counts().sort_index()

In [ ]:
scitcem_only_normal_cells.obs[['filter_dp','filter_alt']].value_counts()

In [ ]:
scitcem_only_normal_cells.obs['filter_alt'].value_counts()

In [ ]:
# 208 iCMS only tumour cell
iCMS_only_tumour_cells = adata_epi[(adata_epi.obs['sample_origin'] == 'tumour\nsample') &
         (adata_epi.obs['scitcem_call'] != 'tumour\n(tumour sample)') &
         (adata_epi.obs['numbat'] != 'tumour\n(tumour sample)') &
         (adata_epi.obs['inferCNV_result'] != 'CNA\n(tumour sample)') &
         (adata_epi.obs['iCMS_scANVI'] != 'normal\n(tumour sample)')]

In [ ]:
iCMS_only_tumour_cells.obs['sample_cap'].value_counts().sort_index()

In [ ]:
iCMS_only_tumour_cells.obs['filter_alt'].value_counts()

In [ ]:
iCMS_only_tumour_cells.obs['clone_opt'].value_counts()

In [ ]:
iCMS_only_tumour_cells.obs['Uhlitz_scANVI'].value_counts().sort_index()

In [ ]:
adata_epi.obs['icms_only_tumour_cells'] = adata_epi.obs.index.isin(iCMS_only_tumour_cells.obs.index.values)

In [ ]:
scv.pl.scatter(adata_epi, basis='umap', color='icms_only_tumour_cells', 
               ncols=1, legend_loc='right margin', size = 2)

In [ ]:
# 67 inferCNV only tumour cell
inferCNV_only_tumour_cells = adata_epi[(adata_epi.obs['sample_origin'] == 'tumour\nsample') &
         (adata_epi.obs['scitcem_call'] != 'tumour\n(tumour sample)') &
         (adata_epi.obs['numbat'] != 'tumour\n(tumour sample)') &
         (adata_epi.obs['inferCNV_result'] == 'CNA\n(tumour sample)') &
         (adata_epi.obs['iCMS_scANVI'] == 'normal\n(tumour sample)')]

In [ ]:
inferCNV_only_tumour_cells.obs['sample_cap'].value_counts().sort_index()

In [ ]:
inferCNV_only_tumour_cells.obs['filter_alt'].value_counts().sort_index()

In [ ]:
inferCNV_only_tumour_cells.obs[['sample_cap','clone_opt', 'filter_alt']].value_counts().sort_index()

In [ ]:
# 57 inferCNV and iCMS only tumour cell
inferCNV_and_iCMS_tumour_cells = adata_epi[(adata_epi.obs['sample_origin'] == 'tumour\nsample') &
         (adata_epi.obs['scitcem_call'] != 'tumour\n(tumour sample)') &
         (adata_epi.obs['numbat'] != 'tumour\n(tumour sample)') &
         (adata_epi.obs['inferCNV_result'] == 'CNA\n(tumour sample)') &
         (adata_epi.obs['iCMS_scANVI'] != 'normal\n(tumour sample)')]

In [ ]:
inferCNV_and_iCMS_tumour_cells.obs['filter_alt'].value_counts().sort_index()

In [ ]:
inferCNV_and_iCMS_tumour_cells.obs[['sample_cap','clone_opt', 'filter_alt']].value_counts().sort_index()

In [ ]:
inferCNV_and_iCMS_tumour_cells.obs['Uhlitz_scANVI'].value_counts().sort_index()

In [ ]:
adata_epi.obs[['inferCNV_result', 'iCMS_scANVI']].value_counts(dropna=False).sort_index()

In [ ]:
upset_df = adata_epi[adata_epi.obs['sample_origin'] == 'tumour\nsample'].obs.copy()

In [ ]:
%%R -i upset_df -w 1200 -h 800 -r 200


# iCMS and inferCNV
df = upset_df %>%
select(iCMS_scANVI, inferCNV_result, patient) %>%
mutate(iCMS = ifelse(iCMS_scANVI != 'normal\n(tumour sample)', TRUE, FALSE),
       inferCNV = ifelse(inferCNV_result == 'CNA\n(tumour sample)', TRUE, FALSE)) 

p = upset(df, c('iCMS', 'inferCNV'), 
      #name = 'tumour calls by iCMS and inferCNV', 
      set_sizes = FALSE, height_ratio = 1/5,
      stripes = upset_stripes(
          colors = 'white'
          ),
      base_annotations = list('Intersection size' = intersection_size(
                                                      counts = FALSE, 
          mapping = aes(fill = patient), width = 0.8) +
                              scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', 
                                                            '#aa40fc', '#8c564b',
                                                            '#e377c2', '#b5bd61', '#17becf', '#aec7e8')) +
                              theme(panel.grid = element_blank(),
                                    axis.ticks = element_line('black'),
                                    axis.line = element_line()) + 
                              plot_theme_font_size(16)
                             )) + 
theme(panel.grid = element_blank())+
plot_theme_font_size(16) 

print(p)

ggsave(file="iCMS_inferCNV_upset_by_pt.svg", width = 10, height = 10, units = "cm", dpi = 300)

# dont use theme_void(), there is something hidden in their code that will show some random texts

In [ ]:
%%R -i upset_df -w 800 -h 700 -r 200


df = upset_df %>%
select(scitcem_call, numbat, MS_status) %>%
mutate(Scitcem = ifelse(scitcem_call == 'tumour\n(tumour sample)', TRUE, FALSE),
       Numbat = ifelse(numbat == 'tumour\n(tumour sample)', TRUE, FALSE)) 

p = upset(df, rev(c('Scitcem', 'Numbat')), 
      #name = 'tumour calls by Scitcem and Numbat', 
      set_sizes = FALSE, height_ratio = 1/5,sort_sets=FALSE,
      stripes = upset_stripes(
          colors = 'white'
          ),
      base_annotations = list('Intersection size' = intersection_size(
                                                      counts = FALSE, mapping = aes(fill = MS_status), width = 0.8) +
                              scale_fill_manual(values = c('#D45B6E', '#EAA945'), name = 'Subtype') +
                              theme(panel.grid = element_blank(),
                                    axis.ticks = element_line('black'),
                                    axis.line = element_line()) + 
                              plot_theme_font_size(15)
                             )) + 
theme(panel.grid = element_blank())+
plot_theme_font_size(15)

print(p)

ggsave(file="Numbat_Scitcem_upset_by_MS.svg", width = 10, height = 10, units = "cm", dpi = 300)

# dont use theme_void(), there is something hidden in their code that will show some random texts

In [ ]:
%%R -i upset_df -w 1200 -h 800 -r 200


df = upset_df %>%
select(scitcem_call, numbat, inferCNV_result, iCMS_scANVI, patient) %>%
mutate(iCMS = ifelse(iCMS_scANVI != 'normal\n(tumour sample)', TRUE, FALSE),
       inferCNV = ifelse(inferCNV_result == 'CNA\n(tumour sample)', TRUE, FALSE),
       Scitcem = ifelse(scitcem_call == 'tumour\n(tumour sample)', TRUE, FALSE),
       Numbat = ifelse(numbat == 'tumour\n(tumour sample)', TRUE, FALSE)) 


p = upset(df, c('Numbat', 'Scitcem', 'inferCNV', 'iCMS'), 
      #name = 'tumour calls by Scitcem and Numbat', 
      set_sizes = FALSE, height_ratio = 1/3.5, sort_sets=FALSE,
      stripes = upset_stripes(
          colors = 'white'
          ),
      base_annotations = list('Intersection size' = intersection_size(
                                                    counts = FALSE, 
                                                    mapping = aes(fill = patient), width = 0.8) +
                               scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', 
                                                            '#aa40fc', '#8c564b',
                                                            '#e377c2', '#b5bd61', '#17becf', '#aec7e8')) +
                              theme(panel.grid = element_blank(),
                                    axis.ticks = element_line('black'),
                                    axis.line = element_line()) + 
                              plot_theme_font_size(15)
                             )) + 
theme(panel.grid = element_blank())+
plot_theme_font_size(15)

print(p)

ggsave(file="All_method_upset_by_pt.svg", width = 14, height = 10, units = "cm", dpi = 300)

# dont use theme_void(), there is something hidden in their code that will show some random texts

In [ ]:
%%R -i upset_df -w 1200 -h 800 -r 200


df = upset_df %>%
select(scitcem_call, numbat, inferCNV_result, iCMS_scANVI, MS_status) %>%
mutate(iCMS = ifelse(iCMS_scANVI != 'normal\n(tumour sample)', TRUE, FALSE),
       inferCNV = ifelse(inferCNV_result == 'CNA\n(tumour sample)', TRUE, FALSE),
       Scitcem = ifelse(scitcem_call == 'tumour\n(tumour sample)', TRUE, FALSE),
       Numbat = ifelse(numbat == 'tumour\n(tumour sample)', TRUE, FALSE)) 


p = upset(df, c('Numbat', 'Scitcem', 'inferCNV', 'iCMS'), 
      #name = 'tumour calls by Scitcem and Numbat', 
      set_sizes = FALSE, height_ratio = 1/3.5, sort_sets=FALSE,
      stripes = upset_stripes(
          colors = 'white'
          ),
      base_annotations = list('Intersection size' = intersection_size(
                                                    counts = FALSE, 
                                                    mapping = aes(fill = MS_status), width = 0.8) +
                               scale_fill_manual(values = c('#D45B6E', '#EAA945'), name = 'Subtype') +
                              theme(panel.grid = element_blank(),
                                    axis.ticks = element_line('black'),
                                    axis.line = element_line()) + 
                              plot_theme_font_size(15)
                             )) + 
theme(panel.grid = element_blank())+
plot_theme_font_size(15)

print(p)

ggsave(file="All_method_upset_by_MS.svg", width = 14, height = 10, units = "cm", dpi = 300)

# dont use theme_void(), there is something hidden in their code that will show some random texts

In [ ]:
cell_identity_list = adata_epi.obs[[
    'cell_identity', 'sample_cap', 'sample_origin','patient','Uhlitz_scANVI', 'Uhlitz_scANVI_noTC']]

In [ ]:
# number of epithelial cells per sample
cell_identity_list['sample_cap'].value_counts().sort_index()

In [ ]:
%%R -i cell_identity_list  -w 600 -h 500 -r 200

# Fig 3d
# bar plot of resolved cell identities in each tumour samples (both agree + more in Methods)


df = cell_identity_list %>% 
filter(sample_origin == 'tumour\nsample') %>%
group_by(sample_cap, cell_identity) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct(sample_cap, cell_identity, sum, sum_sample) %>%
arrange(sample_cap)

CI_order = rev(c('genomically_normal', 'no confident assignment', 'genomically_tumour'))

df$cell_identity = factor(df$cell_identity, levels = CI_order)

p = df %>%
ggplot(aes(x = sum, y = sample_cap, fill = cell_identity)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = rev(c('#1f77b4',
                            '#9b1ee3',
                            '#ff7f0e')),
                   name = 'Cell identity call\n(tumour samples)',
                   labels = rev(c('Genomically\nnormal',
                             'No confident\nassignment',
                              'Genomically\ntumour'))) +
theme_void() +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$sample_cap)), ' (', rev(unique(df$sum_sample)), ')')) +
plot_theme_font_size(8) 

print(p)

ggsave(file="cell_identity_bar.svg", width = 7, height = 5, units = "cm", dpi = 300)

set_infercnv = set(adata_epi[(adata_epi.obs['inferCNV_result'] == 'CNA\n(tumour sample)') & 
                                     (adata_epi.obs['sample_origin'] == 'tumour\nsample')].obs.index)
print(len(set_infercnv))

set_scitcem = set(adata_epi[(adata_epi.obs['scitcem_call'] == 'tumour\n(tumour sample)') & 
                                     (adata_epi.obs['sample_origin'] == 'tumour\nsample')].obs.index)
print(len(set_scitcem))

set_numbat = set(adata_epi[(adata_epi.obs['numbat'] == 'tumour\n(tumour sample)')& 
                                     (adata_epi.obs['sample_origin'] == 'tumour\nsample')].obs.index)
print(len(set_numbat))

set_icms_scANVI = set(adata_epi[(adata_epi.obs['iCMS_scANVI'] != 'normal\n(tumour sample)')& 
                                     (adata_epi.obs['sample_origin'] == 'tumour\nsample')].obs.index)
print(len(set_icms_scANVI))

labels = venn.get_labels([set_icms_scANVI, set_scitcem, set_numbat, set_infercnv], fill = ['number'])
fig, ax = venn.venn4(labels, names = ['iCMS' ,'Scitcem', 'Numbat', 'inferCNV'])
#fig.suptitle('agreement on tumour cells (tumour sample only)', fontsize=24)
#ax.legend(bbox_to_anchor=(1.04, 1))
#ax.legend(frameon = False)
plt.legend([arr1a, arr2, arr3], ['u','v','u+v'])


In [ ]:
%%R -i upset_df -w 1600 -h 1200 -r 200

method_colors = c('#f57379','#f5d107', '#228b22', '#47a2ed')

df = upset_df %>%
select(scitcem_call, numbat, inferCNV_result, iCMS_scANVI,MS_status) %>%
mutate(iCMS = ifelse(iCMS_scANVI != 'normal\n(tumour sample)', TRUE, FALSE),
       inferCNV = ifelse(inferCNV_result == 'CNA\n(tumour sample)', TRUE, FALSE),
       Scitcem = ifelse(scitcem_call == 'tumour\n(tumour sample)', TRUE, FALSE),
       Numbat = ifelse(numbat == 'tumour\n(tumour sample)', TRUE, FALSE)) 


p=ggvenn(df, stroke_size = 0.2, set_name_size = 6, show_percentage = FALSE,
       fill_alpha = 0.5,
       set_name_color = method_colors,
       fill_color = method_colors) +
plot_theme_font_size(20)+
theme(axis.text = element_blank(),
     axis.title = element_blank())

print(p)

ggsave(file="venn.svg", width = 20, height = 20, units = "cm", dpi = 300)

In [ ]:
# Fig 3e
# UMAP of resolved cell identity

fig = sc.pl.umap(adata_epi, color=['cell_identity'], 
               ncols=1, legend_loc='right margin', size = 2,
              title = '', frameon = False, return_fig=True)

In [ ]:
# UMAP of resolved cell identity
fig = sc.pl.umap(adata_epi_noTC_cleanCI, color=['cell_identity'], 
               ncols=1, legend_loc='right margin', size = 2,
              title = '', frameon = False, return_fig=True)

In [ ]:
fig = sc.pl.umap(adata_epi_noTC_cleanCI, color=['PLA2G2A'], 
               ncols=1, legend_loc='right margin', size = 2,
               frameon = False, return_fig=True)

In [ ]:
fig = sc.pl.umap(adata_epi_noTC_cleanCI, color=['MUC1', 'MUC5B', 'OLFM4', 'WNT2'], 
               ncols=2, legend_loc='right margin', size = 2,
               frameon = False, return_fig=True)

### Figure 4
- Tumour-like == genomicalyl tumour and remove TCs in normal sample and genomically normal
- no 'no confident assignment' cells

In [ ]:
adata_epi.obs['Uhlitz_scANVI_noTC'].cat.categories

In [ ]:
cell_identity_list_clean = adata_epi_noTC_cleanCI.obs[[
    'cell_identity', 'sample_cap', 'sample_origin','patient','Uhlitz_scANVI', 'Uhlitz_scANVI_noTC']]

In [ ]:
# Fig 4a
# UMAP with cell type annotation (epi cells)

fig = sc.pl.umap(adata_epi_noTC_cleanCI, color=['Uhlitz_scANVI_noTC'], 
               ncols=1, legend_loc='right margin', size = 2, title = 'Epithelial cell type',
               frameon = False, return_fig=True)

In [ ]:
%%R -i cell_identity_list_clean -w 700 -h 500 -r 200

# bar plot of resolved cell identities in each tumour samples (both agree + more in Methods)
# clean identity

df = cell_identity_list_clean %>% 
filter(sample_origin == 'tumour\nsample') %>%
group_by(sample_cap, cell_identity) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct(sample_cap, cell_identity, sum, sum_sample) %>%
arrange(sample_cap)

CI_order = c('genomically_normal', 'genomically_tumour')
df$cell_identity = factor(df$cell_identity, levels = CI_order)

p = df %>%
ggplot(aes(x = sum, y = sample_cap, fill = cell_identity)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#1f77b4',
                            '#ff7f0e'),
                   name = 'Cell identity call\n(tumour samples)',
                   labels = c('genomically\nnormal',
                              'genomically\ntumour')) +
theme_void() +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,labels = paste0(rev(unique(df$sample_cap)), ' (', rev(unique(df$sum_sample)), ')')) +
plot_theme_font_size(8) 

print(p)

ggsave(file="cell_identity_solved_by_sample_bar.svg", width = 7, height = 5, units = "cm", dpi = 300)

In [ ]:
%%R -i cell_identity_list_clean,Uhlitz_scANVI_noTC_color_list_after -w 1200 -h 500 -r 200

# cell type differences in genomically normal vs normal samples

df = cell_identity_list_clean %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>%
group_by(sample_cap, cell_identity, Uhlitz_scANVI_noTC) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct() %>%
arrange(desc(sample_cap))

df$Uhlitz_scANVI_noTC = factor(df$Uhlitz_scANVI_noTC, rev(levels(df$Uhlitz_scANVI_noTC)))


p = df %>%
ggplot(aes(x = sum, y = patient, fill = Uhlitz_scANVI_noTC)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = rev(Uhlitz_scANVI_noTC_color_list_after),
                 name = 'Epithelial cell type')+
theme_void() + 
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Patient\n') +
facet_grid(cols = vars(cell_identity)) +
scale_y_discrete(limits = rev) +
scale_x_continuous(n.breaks = 3) +
plot_theme_font_size(8) 

print(p)

ggsave(file="cell_type_GN_vs_NN_pt_bar.svg", width = 12, height = 5, units = "cm", dpi = 300)

In [ ]:
%%R -i cell_identity_list_clean,Uhlitz_scANVI_noTC_color_list_after -w 1200 -h 500 -r 200

# Fig 4b
# cell type differences in genomically normal vs normal samples
suppressMessages(library(tidyverse))



df = cell_identity_list_clean %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>%
group_by(cell_identity, Uhlitz_scANVI_noTC) %>%
mutate(sum=n()) %>%
distinct(cell_identity, Uhlitz_scANVI_noTC, sum) 


df$Uhlitz_scANVI_noTC = factor(df$Uhlitz_scANVI_noTC, rev(levels(df$Uhlitz_scANVI_noTC)))

p = df %>% 
ggplot(aes(x = sum, y = cell_identity, fill = Uhlitz_scANVI_noTC)) +
geom_col(width = 0.8, position = 'fill') +
theme_void() + 
theme(axis.title.y = element_text(angle = 90)) +
scale_fill_manual(values = rev(Uhlitz_scANVI_noTC_color_list_after),name = 'Epithelial cell type') +
labs(x = '\nProportion\n', 
     y = 'Cell identity\n') +
guides(fill=guide_legend(ncol=2)) +
scale_x_continuous(n.breaks = 3) +
plot_theme_font_size(8) 

print(p)

ggsave(file="GN_NN_cell_type_bar.svg", width = 15, height = 4, units = "cm", dpi = 300)

%%R -i cell_identity_list -w 1800 -h 1200 -r 200

suppressMessages(library(tidyverse))
suppressMessages(library('cowplot'))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm"),
                   strip.text.x = element_text(size = 12)) 

table(cell_identity_list[c('cell_identity', 'Uhlitz_scANVI')]) %>%
as.data.frame() %>%
group_by(Uhlitz_scANVI) %>%
mutate(Percentage = round(Freq/sum(Freq)*100,0),Sum = sum(Freq)) %>% 
group_by(cell_identity) %>% mutate(Percentage_sample = round(Freq/sum(Freq)*100,0),Sum_sample = sum(Freq)) %>%
ggplot(aes(x = Uhlitz_scANVI, y = cell_identity)) + 
geom_tile(aes(fill = Percentage_sample), color = 'black') +
geom_text(aes(label = paste0(Freq, '\n', 
                             Percentage, '%\n',
                             Percentage_sample, '%'))) +
scale_fill_gradient(low = "grey95", high = "red", name = '% per cell identity') +
plot_theme + 
#coord_equal() +
theme(plot.title = element_text(hjust = 0.5),
      rect = element_blank(),
      axis.ticks = element_blank(),
      axis.text.x = element_text(angle = 90)) +
#guides(fill = guide_legend(title = "Count")) +
labs(title = '\nNumber of cell in each group\n',
     x = 'Cell type (including TCs)',
     y = 'Cell identity\n')

In [ ]:
%%R -i cell_identity_list,cell_type_list -w 1400 -h 800 -r 200

# Fig 4c
# Statistical testing of cell type differences in genomically normal vs normal samples

df = cell_identity_list %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>%
filter(!patient %in% c('P08', 'P21', 'P26', 'P35') ) 

df$cell_identity = factor(df$cell_identity, levels = c('normal_sample', 'genomically_normal'))

summary_model = list()
summary_p = list()

for(i in cell_type_list){
    glmer_model = glmer(Uhlitz_scANVI == i ~ cell_identity + (1|patient), 
             family="binomial", data = df)
    summary_model[[i]] = summary(glmer_model)
    summary_p[[i]] = data.frame('Uhlitz_scANVI' = i, 
           'p' = summary(glmer_model)$coefficients['cell_identitygenomically_normal','Pr(>|z|)'])
}
# boundary (singular) fit: see help('isSingular')
# means random effects are very small

glmer_p_stats = bind_rows(summary_p) %>% 
filter(!Uhlitz_scANVI %in% c('TC1','TC2','TC3','TC4')) %>%
mutate(loc = seq(1,7,1), p = signif(p,2)) %>%
mutate(star = case_when(p < 0.05 ~ '< 0.05'))

print(glmer_p_stats)

table(cell_identity_list[c('Uhlitz_scANVI', 'patient', 'cell_identity')]) %>%
as.data.frame() %>%
filter(cell_identity %in% c('genomically_normal', 'normal_sample')) %>%
filter(!patient %in% c('P08', 'P21', 'P26', 'P35') ) %>%
filter(!Uhlitz_scANVI %in% c('TC1','TC2','TC3','TC4')) %>%
group_by(cell_identity, patient) %>% 
mutate(frac_sample = Freq/sum(Freq)) %>% 
ggplot(aes(x = Uhlitz_scANVI, y = frac_sample)) +
geom_boxplot(outlier.color = 'white', aes(color = cell_identity)) +
geom_point(position=position_jitterdodge(), aes(color = cell_identity), size = 1)+
theme_bw() + 
theme(axis.text.x = element_text(angle = 90),
      panel.grid = element_blank()) +
labs(x = 'Epithelial cell types',
     y = 'Fraction per sample\n') +
scale_color_manual(values = c('#1f77b4', '#808080'), name = 'Cell identity') +
geom_bracket(aes(xmin = loc-0.25, xmax = loc+0.25, label = p), data = glmer_p_stats, 
             y.position = c(0.45, 0.4, 0.2, 0.4, 0.45, 0.55, 0.25), size = 0.2, label.size = 3) +
ylim(0,0.65) +
plot_theme_font_size(8) 

ggsave(file="GN_NN_cell_type_bar.svg", width = 15, height = 4, units = "cm", dpi = 300)

In [ ]:
# Fig 4d
# diffusion map

fig = sc.pl.diffmap(adata_epi_noTC_cleanCI, color=['Uhlitz_scANVI_noTC'], 
               ncols=1, legend_loc='right margin', size = 2, title = 'Diffusion map dim 1 and 2',
               frameon = False, return_fig=True)


In [ ]:
adata_epi_noTC_cleanCI.obs[['DC1', 'DC2', 'DC3', 'DC4']]= adata_epi_noTC_cleanCI.obsm['X_diffmap'][:,1:5].copy()

In [ ]:
order_start_with_tumour_noTC = [
 'Tumour-like',
 'Stem',
 'Stem/TA',
 'Immature Goblet',
 'Goblet',
 'Enterocyte progenitor',
 'Enterocytes',
 'Tuft']

In [ ]:
order_start_with_stem_noTC = ['Stem',
 'Stem/TA',
 'Immature Goblet',
 'Goblet',
 'Enterocyte progenitor',
 'Enterocytes',
 'Tuft']

In [ ]:
with plt.rc_context({'ytick.major.size':18, 'axes.titlesize':18}):
    plt.figure(figsize = (8/2.54,5/2.54))

    g = sns.JointGrid(data = adata_epi_noTC_cleanCI.obs, x = 'DC1', y = 'DC2', hue = 'Uhlitz_scANVI_noTC',
                      hue_order=order_start_with_tumour_noTC,
                      palette=[sample_color_map[key] for key in order_start_with_tumour_noTC],
                      height = 9, ratio = 3)
    g.plot_joint(sns.scatterplot, s = 2, edgecolor = None)
    g.plot_marginals(sns.histplot, kde = False, binwidth=0.001, edgecolor = 'black', linewidth = 0.3, alpha =1)

    sns.move_legend(g.ax_joint, "upper left", bbox_to_anchor=(1.5, 1), title = 'Epithelial cell type', 
                    frameon=False, title_fontsize = 14)

    #g.ax_joint.legend_.remove()
;

In [ ]:
adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['cell_identity'] == 'genomically_normal'].obs['DC2'].min()

In [ ]:
adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['cell_identity'] == 'genomically_normal'].obs['DC2'].max()

adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] == 'normal_sample') & 
         (adata_epi_noTC_cleanCI.obs['DC2'] < -0.006)].obs[['sample', 'Uhlitz_scANVI_noTC']]
#use in CellChat obj

In [ ]:
np.arange(-0.007,0.012,0.001)

In [ ]:
with plt.rc_context({'ytick.major.size':16, 'axes.titlesize':18}):
    plt.figure(figsize = (8,5))
    ax = sns.histplot(adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['cell_identity'] == 'genomically_normal'].obs, 
                      x = 'DC2', hue = 'Uhlitz_scANVI_noTC', binwidth=0.001, stat = 'count',
                      legend='right', multiple="stack", 
                      palette=[sample_color_map[key] for key in order_start_with_stem_noTC],
                      hue_order=order_start_with_stem_noTC, edgecolor = 'black')
    sns.move_legend(ax, "upper right", bbox_to_anchor=(1.4, 1.0), title='Cell type', frameon=False)
    plt.title('Diffusion map dim 2 distribution of\ngenomically normal cells\n')
    plt.ylabel('Normalised count')

# Dig out how many cells per bin
# can only use with stack due to normalised count if use multiple = fill
ary = []

print(len(ax.containers))

for i in ax.containers:
    ary = np.append(ary,i.datavalues)


number_cell_per_bin = [str(int(sum(x))) for x in zip(*ary.reshape(7,18))]
print(number_cell_per_bin)

In [ ]:
# Fig 4e
# DC2 of genomically normal and normal cells
with plt.rc_context({'ytick.major.size':16, 'axes.titlesize':18}):
    plt.figure(figsize = (8,5))
    ax = sns.histplot(adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['cell_identity'] == 'genomically_normal'].obs, 
                      x = 'DC2', hue = 'Uhlitz_scANVI_noTC', binwidth=0.001, stat = 'count',
                      legend='right', multiple="fill", 
                      palette=[sample_color_map[key] for key in order_start_with_stem_noTC],
                      hue_order=order_start_with_stem_noTC, edgecolor = 'black')
    sns.move_legend(ax, "upper right", bbox_to_anchor=(1.45, 1.0), title='Epithelial cell type', frameon=False,
                   title_fontsize = 15)
    #plt.title('Diffusion map dim 2 distribution of\ngenomically normal cells\n')
    plt.ylabel('Normalised count', size = 15)
    plt.xlabel('Diffusion map dimension 2', size = 15)
    
    #x = np.arange(-0.0061,0.018,0.00101)
    #y = [1.05]*18
    #for i, text in enumerate(number_cell_per_bin):
    #    ax.text(x[i], y[i], text, fontsize=10, ha = 'right',va = 'center')
    

In [ ]:
adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['sample_origin'] == '\nnormal\nsample\n'].obs['DC2'].min()

In [ ]:
adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['sample_origin'] == '\nnormal\nsample\n'].obs['DC2'].max()

In [ ]:
with plt.rc_context({'ytick.major.size':16, 'axes.titlesize':18}):
    plt.figure(figsize = (8,5))
    ax = sns.histplot(adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['sample_origin'] == '\nnormal\nsample\n'].obs, 
                      x = 'DC2', hue = 'Uhlitz_scANVI_noTC', binwidth=0.001, stat = 'count',
                      legend='right', multiple="stack", 
                      palette=[sample_color_map[key] for key in order_start_with_stem_noTC],
                      hue_order=order_start_with_stem_noTC, edgecolor = 'black')
    sns.move_legend(ax, "upper right", bbox_to_anchor=(1.45, 1.0), title='Cell type', frameon=False)
    plt.title('Diffusion map dim 2 distribution of\ncells from normal samples\n')
    plt.ylabel('Normalised count')
    
ary = []

print(len(ax.containers))
for i in ax.containers:
    ary = np.append(ary,i.datavalues)
    
number_cell_per_bin = [str(int(sum(x))) for x in zip(*ary.reshape(7,19))] 
print(number_cell_per_bin)

In [ ]:
# Figure 4e
with plt.rc_context({'ytick.major.size':16, 'axes.titlesize':18}):
    plt.figure(figsize = (8,5))
    ax = sns.histplot(adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['sample_origin'] == '\nnormal\nsample\n'].obs, 
                      x = 'DC2', hue = 'Uhlitz_scANVI_noTC', binwidth=0.001, stat = 'count',
                      legend='right', multiple="fill", 
                      palette=[sample_color_map[key] for key in order_start_with_stem_noTC],
                      hue_order=order_start_with_stem_noTC, edgecolor = 'black')
    sns.move_legend(ax, "upper right", bbox_to_anchor=(1.45, 1.0), title='Epithelial cell type', frameon=False,
                   title_fontsize = 15)
    #plt.title('Diffusion map dim 2 distribution of\ncells from normal samples\n')
    plt.ylabel('Normalised count', size = 15)
    plt.xlabel('Diffusion map dimension 2', size = 15)

    
    #x = np.arange(-0.0068,0.013,0.001)
    #y = [1.04]*19
    #for i, text in enumerate(number_cell_per_bin):
    #    ax.text(x[i], y[i], text, fontsize=8, ha = 'center',va = 'center')
    

In [ ]:
# Figure 
fig = sc.pl.umap(adata_epi_noTC_cleanCI, color=['ct_pseudotime'], 
               ncols=1, legend_loc='right margin', size = 2, title = '',
               frameon = False, return_fig=True, color_map='viridis_r')


In [ ]:
fig = sc.pl.diffmap(adata_epi_noTC_cleanCI, color=['ct_pseudotime'], 
               ncols=1, legend_loc='right margin', size = 2, title = '',
               frameon = False, return_fig=True, color_map='viridis_r')


In [ ]:
# cytoTRACE
with plt.style.context('./plt_style'):
    g=sns.FacetGrid(adata_epi_noTC_cleanCI.obs, 
                    col = 'Uhlitz_scANVI_noTC', hue = 'cell_identity', 
                    height = 4, aspect = 5/9, despine = False, sharey = True, 
                    #col_order = new_order_noTC, 
                    palette = ['#ff7f0e',
                               '#1f77b4','#808080']) # a darker grey for normal

    g.map_dataframe(sns.violinplot, x = 'cell_identity', y = 'ct_pseudotime', width = 0.7)
    g.figure.subplots_adjust(wspace=0)
    
    g.set_xticklabels(rotation = 90)
    g.set_titles('{col_name}\n')
    g.set(ylim=(0, 1))
    g.set_xlabels('')
    g.set_ylabels('Cytoscape pseudotime\n', loc = 'top')
    
    g.fig.text(x = 0.5, y = -1, horizontalalignment = 'center', s = 'Cell identity', size = 24)
    
    axes = g.axes.flatten()
    axes[6].set_title('Enterocyte\nprogenitor\n')
    axes[7].set_title('Enterocyte\n')
    axes[4].set_title('Immature\ngoblet\n')
;    

#Fig4f
#cytoTRACE without genomically tumour
with plt.style.context('./plt_style'):
    g=sns.FacetGrid(adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['cell_identity'] != 'genomically_tumour'].obs, 
                    col = 'Uhlitz_scANVI_noTC', hue = 'cell_identity', 
                    height = 4, aspect = 5/9, despine = False, sharey = True, 
                    #col_order = new_order_noTC, 
                    palette = [#'#ff7f0e',
                               '#1f77b4','#808080']) # a darker grey for normal

    g.map_dataframe(sns.violinplot, x = 'cell_identity', y = 'ct_pseudotime', width = 0.7)
    g.figure.subplots_adjust(wspace=0)
    
    g.set_xticklabels(rotation = 90)
    g.set_titles('{col_name}\n')
    g.set(ylim=(0, 1))
    g.set_xlabels('')
    g.set_ylabels('Cytoscape pseudotime\n', loc = 'top')
    
    g.fig.text(x = 0.5, y = -1, horizontalalignment = 'center', s = 'Cell identity', size = 24)
    
    axes = g.axes.flatten()
    axes[5].set_title('Enterocyte\nprogenitor\n')
    axes[6].set_title('Enterocyte\n')
    axes[3].set_title('Immature\ngoblet\n')
;    

In [ ]:
cyto_list = adata_epi_noTC_cleanCI.obs[['cell_identity', 'ct_pseudotime', 'Uhlitz_scANVI_noTC', 'louvain']]

In [ ]:
%%R -i cyto_list,Uhlitz_scANVI_noTC_color_list -w 3000 -h 900 -r 200

p = cyto_list %>% 
mutate(Uhlitz_scANVI_noTC = dplyr::recode(Uhlitz_scANVI_noTC, 'Tumour-like' =  'Tumour- like')) %>%
ggplot(aes(x = cell_identity, y = ct_pseudotime)) +
geom_violin(aes(fill = Uhlitz_scANVI_noTC), width = 0.8) +
geom_boxplot(width = 0.2) +
#geom_jitter(shape = 16, position = position_jitter(0.3), size = 0.01) +
theme_void()+
plot_theme_font_size(15) +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90),
      axis.ticks.y = element_line('black', linewidth = 1),
      axis.ticks.length=unit(0.1, "cm"),
      axis.text.y = element_text(margin = margin(t = 0, r = 7, b = 0, l = 0)),
      axis.line = element_line()) +
scale_fill_manual(values = Uhlitz_scANVI_noTC_color_list, name = 'Epithelial cell type') +
facet_grid(cols = vars(Uhlitz_scANVI_noTC), space = 'free', scale = 'free', 
          labeller = label_wrap_gen(width = 10)) + 
labs(x = 'Cell identity',
     y = 'Cytotrace pseudotime\n') +
scale_y_continuous(n.breaks = 3)

print(p)

ggsave(file="ct_cell_type_violin.svg", width = 38, height = 12, units = "cm", dpi = 300)


In [ ]:
%%R -i cyto_list,order_start_with_stem_noTC
 
for(i in order_start_with_stem_noTC){
    df = cyto_list %>% 
    filter(Uhlitz_scANVI_noTC == i) %>%
    filter(cell_identity != 'genomically_tumour')
    
    print(i)
    print(pairwise.t.test(df$ct_pseudotime, df$cell_identity, p.adj='bonferroni'))
   
}


In [ ]:
%%R -i cyto_list,order_start_with_stem_noTC

 
for(i in order_start_with_stem_noTC){
    df = cyto_list %>%
    filter(Uhlitz_scANVI_noTC == i) %>%
    filter(cell_identity != 'genomically_tumour')
    
    print(i)
    
    print(t.test(ct_pseudotime ~ cell_identity, alternative = "two.sided", 
       data = df,
       conf.level = .95, 
       paired = FALSE))
}

In [ ]:
%%R -i cyto_list

# contrast in linear model
library('emmeans')

cyto_lm  = cyto_list %>% filter(cell_identity != 'genomically_tumour') %>%
lm(ct_pseudotime ~ Uhlitz_scANVI_noTC + cell_identity, data = .)
print(summary(cyto_lm))

pairs(emmeans(cyto_lm, ~cell_identity))


In [ ]:
%%R -i cyto_list


for(i in order_start_with_stem_noTC){
    df = cyto_list %>%
    filter(Uhlitz_scANVI_noTC == i) %>%
    filter(cell_identity != 'genomically_tumour')
    
    print(i)
    
    df$cell_identity = relevel(df$cell_identity, ref = 'normal_sample')
    lm_Uhlitz = lm(ct_pseudotime ~ cell_identity, data = df)
    print(summary(lm_Uhlitz))
    
    print(pairs(emmeans(lm_Uhlitz, ~ cell_identity), reverse = TRUE))
}


In [ ]:
%%R -i cyto_list

# two sample wilcox
library('rstatix')

wilcox_test(data = cyto_list, ct_pseudotime ~ cell_identity, ref.group = 'genomically_normal') %>% as.data.frame()

%%R -i cyto_list

#two sample wilcox

for(i in order_start_with_stem_noTC){
    df = cyto_list %>%
    filter(Uhlitz_scANVI_noTC == i) %>%
    filter(cell_identity != 'genomically_tumour')
    
    print(i)
    
    print(wilcox.test(data = df, ct_pseudotime ~ cell_identity, paired = FALSE, alternative = "two.sided"))
}


In [ ]:
%%R -i cyto_list

# two sample wilcox with multiple testing correction

for(i in order_start_with_stem_noTC){
    
    df = cyto_list %>%
    filter(Uhlitz_scANVI_noTC == i) %>%
    filter(cell_identity != 'genomically_tumour')
    
    print(i)
    
    print(pairwise.wilcox.test(df$ct_pseudotime, df$cell_identity, paired = FALSE, p.adjust.method = "bonf"))
}


In [ ]:
# cluster pseudotime
with plt.style.context('./plt_style'):
    g=sns.FacetGrid(adata_epi_noTC_cleanCI.obs, 
                    col = 'louvain', hue = 'sample_origin', 
                    height = 4, aspect = 3/9, despine = False, sharey = True, 
                    #col_order = new_order_noTC, 
                    palette = ['#ff7f0e',
                               #'#1f77b4',
                               '#808080']) # a darker grey for normal

    g.map_dataframe(sns.violinplot, x = 'sample_origin', y = 'ct_pseudotime', width = 0.7)
    g.figure.subplots_adjust(wspace=0)
    
    g.set_xticklabels(rotation = 90)
    g.set_titles('{col_name}\n')
    g.set(ylim=(0, 1))
    g.set_xlabels('')
    g.set_ylabels('Cytoscape pseudotime\n', loc = 'top')
    
    g.fig.text(x = 0.5, y = -1, horizontalalignment = 'center', s = 'sample_origin', size = 24)
    
    axes = g.axes.flatten()
    #axes[5].set_title('Enterocyte\nprogenitor\n')
    #axes[6].set_title('Enterocyte\n')
    #axes[3].set_title('Immature\ngoblet\n')

In [ ]:
%%R -i cyto_list -w 2200 -h 900 -r 200

suppressMessages(library(tidyverse))

cyto_list %>% 
mutate(Uhlitz_scANVI_noTC = dplyr::recode(Uhlitz_scANVI_noTC, 'Tumour-like' =  'Tumour- like')) %>%
ggplot(aes(x = cell_identity, y = ct_pseudotime)) +
geom_violin(aes(fill = cell_identity)) +
#geom_boxplot(width = 0.2) +
#geom_jitter(shape = 16, position = position_jitter(0.3), size = 0.01) +
theme_void()+
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90),
      strip.text.x = element_text(size = 10)) +
scale_fill_manual(values = c('#ff7f0e', 
                             '#1f77b4','#808080')) +
facet_grid(cols = vars(louvain), space = 'free', scale = 'free', 
          labeller = label_wrap_gen(width = 10)) + 
labs(x = 'Cell identity',
     y = 'Cytotrace pseudotime\n') +
plot_theme_font_size(15)


In [ ]:
%%R -i cyto_list,Uhlitz_scANVI_noTC_color_list -w 2000 -h 800 -r 200

cyto_list %>% 
filter(louvain %in% c('9')) %>%
mutate(Uhlitz_scANVI_noTC = dplyr::recode(Uhlitz_scANVI_noTC, 'Tumour-like' =  'Tumour- like')) %>%
ggplot(aes(x = cell_identity, y = ct_pseudotime)) +
geom_violin(aes(fill = Uhlitz_scANVI_noTC)) +
theme_void()+
plot_theme_font_size(15) +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90),
      strip.text.x = element_text(size = 10),
      axis.line = element_line(),
      axis.ticks = element_line()) +
scale_fill_manual(values = c('#AA0000',
 '#00CCFF',
 '#0072b1', '#990099','#999900',
 '#336600')) +
facet_grid(cols = vars(Uhlitz_scANVI_noTC),
          labeller = label_wrap_gen(width = 10)) + 
labs(x = 'Cell identity',
     y = 'Cytotrace pseudotime\n') 


In [ ]:
%%R -i cyto_list,Uhlitz_scANVI_noTC_color_list -w 2000 -h 800 -r 200


cyto_list %>% 
filter(Uhlitz_scANVI_noTC == 'Stem') %>%
ggplot(aes(x = cell_identity, y = ct_pseudotime)) +
geom_violin(aes(fill = louvain)) +
theme_void()+
plot_theme_font_size(15) +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90),
      strip.text.x = element_text(size = 10),
      axis.line = element_line(),
      axis.ticks = element_line(),
      panel.grid.major.y =  element_line(color = 'grey', size = 0.25)) +
facet_grid(cols = vars(louvain),
          labeller = label_wrap_gen(width = 10)) + 
labs(x = 'Cell identity',
     y = 'Cytotrace pseudotime\n') +
ylim(0,1)


In [ ]:
adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['Uhlitz_scANVI_noTC'] == 'Stem'].obs['louvain'].value_counts().sort_index()

In [ ]:
%%R -i cyto_list,Uhlitz_scANVI_noTC_color_list -w 3000 -h 1600 -r 200

# clusters that have some Stem and Stem/TA
suppressMessages(library(tidyverse))


cyto_list %>% 
mutate(Uhlitz_scANVI_noTC = dplyr::recode(Uhlitz_scANVI_noTC, 'Tumour-like' =  'Tumour- like')) %>%
ggplot(aes(x = cell_identity, y = ct_pseudotime)) +
geom_violin(aes(fill = Uhlitz_scANVI_noTC)) +
#geom_boxplot(width = 0.2) +
#geom_jitter(shape = 16, position = position_jitter(0.3), size = 0.01) +
theme_void()+
plot_theme_font_size(15) +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90),
      strip.text = element_text(size = 10),
      axis.line = element_line(),
      axis.ticks = element_line(), 
      panel.border = element_rect(color = 'black', fill = NA, size = 0.5), 
      panel.grid.major.y = element_line(color = 'grey', size = 0.25)) +
scale_fill_manual(values = Uhlitz_scANVI_noTC_color_list) +
facet_grid(cols = vars(louvain), rows = vars(Uhlitz_scANVI_noTC),
          labeller = label_wrap_gen(width = 10)) + 
labs(x = '\nCell identity',
     y = 'Cytotrace pseudotime\n') 


#### export table for Markus

table_for_Markus = adata_epi.obs[['sample', 'sample_origin', 'filter_dp', 'iCMS_scANVI', 'inferCNV_result', 'scitcem_call', 
                                  'scitcem_p', 'numbat', 'p_cnv', 'cell_identity', 'Uhlitz_scANVI']]

table_for_Markus['sample_origin'].cat.rename_categories({'\nnormal\nsample\n':'normal_sample', 
                                                     'tumour\nsample':'tumour_sample'}, inplace =True)

table_for_Markus['iCMS_scANVI'].cat.rename_categories({'iCMS2\n(tumour sample)':'iCMS2 (tumour sample)', 
                                                      'iCMS3\n(tumour sample)':'iCMS3 (tumour sample)',
                                                       'normal\n(tumour sample)':'normal (tumour sample)'}, 
                                                      inplace =True)

table_for_Markus['inferCNV_result'].cat.rename_categories({'CNA\n(tumour sample)':'CNA (tumour sample)', 
                                                        'CNN\n(tumour sample)':'CNN (tumour sample)'}, 
                                                          inplace =True)

table_for_Markus['scitcem_call'].cat.rename_categories({'tumour\n(tumour sample)':'tumour (tumour sample)', 
                                                        'normal\n(tumour sample)':'normal (tumour sample)'}, 
                                                          inplace =True)

table_for_Markus['numbat'].cat.rename_categories({'tumour\n(tumour sample)':'tumour (tumour sample)', 
                                                        'normal\n(tumour sample)':'normal (tumour sample)'}, 
                                                          inplace =True)

table_for_Markus.columns

table_for_Markus.rename(columns={'iCMS_scANVI':'iCMS',
                                 'inferCNV_result':'inferCNV',
                                 'scitcem_call': 'Scitcem',
                                 'scitcem_p':'Scitcem_p',
                                 'numbat': 'Numbat',
                                 'p_cnv': 'Numbat_p',
                                 'cell_identity': 'Numbat_Scitcem_Resolved',
                                 'filter_dp':'Total_SNV',
                                 'Uhlitz_scANVI' : 'Cell_type'}, inplace=True)

table_for_Markus.to_csv(Path(new_data_folder)/'202311_four_tools_result_and_Numbat_Scitcem_resolved_cell_identity.csv')

adata_all.obs = adata_all.obs.merge(adata_epi.obs['cell_identity'], left_index = True,
                    right_index = True, how='left').set_axis(adata_all.obs.index)

adata_all.obs['cell_identity'] = adata_all.obs['cell_identity'].fillna('normal_sample')

adata_all.obs['cell_identity'].value_counts(dropna=False)

#another table
allcell_table_for_Markus = adata_all.obs[['sample', 'sample_origin', 'celltype_1a', 'cell_type_level2', 'cell_identity']]

allcell_table_for_Markus.to_csv(Path(new_data_folder)/'202311_epi_imm_str_cell_type_and_cell_identity.csv')

### Figure 5

In [ ]:
all_cell_list = adata_all_noTC_cleanCI.obs[['patient', 'sample_cap',
                                            'celltype_1a','cell_type_level2', 'sample_origin']]

In [ ]:
%%R -i cell_identity_list_clean,all_cell_list,removed_TC_id -w 1000 -h 500 -r 200

# Fig. 5a 
# proportion of each cell compartment (epi, imm, str) of each sample
suppressMessages(library(tidyverse))


df = all_cell_list %>%
merge(cell_identity_list_clean[c('cell_identity', 'Uhlitz_scANVI_noTC')], by = 0, all.x = TRUE) %>%
#filter(Row.names %notin% removed_TC_id) %>%
mutate(cell_type_merge = coalesce(cell_identity, celltype_1a)) %>%
group_by(sample_cap, cell_type_merge) %>%
mutate(sum=n()) %>%
group_by(sample_cap) %>%
mutate(sum_sample = n()) %>%
distinct(sample_cap,cell_type_merge, sum, sum_sample, patient, sample_origin)

df$cell_type_merge = factor(df$cell_type_merge, levels = c('imm', 'str', 'normal_sample',
                                                             'genomically_normal', 
                                                             'genomically_tumour'))

p = df %>%
ggplot(aes(x = sum, y = patient, fill = cell_type_merge)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#ED5D47',
                            '#4EAC57',
                             '#144FAC',
                            '#1f77b4',
                            '#ff7f0e'),
                   name = 'Cell identity',
                   labels = c('Immune',
                             'Stromal', 
                              'Epithelial',
                             'Genomically\nnormal',
                             'Genomically\ntumour')) +
theme_void() + 
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90),
      panel.spacing = unit(10, "pt")) +
labs(x = 'Proportion', 
     y = '') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$patient)))) +
facet_grid(cols = vars(sample_origin), 
           labeller = as_labeller(c('normal' = 'Normal\nsample', 'tumour' = 'Tumour\nsample')))
print(p)

ggsave(file="cell_identity_and_TME_normal_tumour_sample.svg", width = 8, height = 5, units = "cm", dpi = 300)


In [ ]:
# Figure 5b pathway activity


In [ ]:
# Figure 5 expressed receptors on epi > go to 20231122_ligand...notebook

In [ ]:
adata_all_noTC_cleanCI.obs['celltype_1a'].cat.categories

In [ ]:
adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'].obs['cell_type_level2'].cat.categories

In [ ]:
sample_color_map_for_all = sample_color_map.copy()

sample_color_map_for_all.update(dict(zip(adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'].obs['cell_type_level2'].cat.categories, 
         ['#7d8df0', '#335B8E', '#79c9c9', '#67C9FF',
          '#CE9486', '#8f7022', '#cf7708', '#B5B867',
          '#DEB058', '#899DA4','#1aa375', '#AA82A7', '#c44545'
         ])))

In [ ]:
sample_color_map_grey = dict(zip(adata_epi_noTC_cleanCI.obs['Uhlitz_scANVI_noTC'].cat.categories, 
                            ['#d3d3d3']*8))

In [ ]:
sample_color_map_grey.update(dict(zip(adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'].obs['cell_type_level2'].cat.categories, 
         ['#7d8df0', '#335B8E', '#79c9c9', '#67C9FF',
          '#CE9486', '#8f7022', '#cf7708', 
          '#B5B867',
          '#DEB058', '#899DA4','#1aa375', '#AA82A7', '#c44545'
         ])))

In [ ]:
sample_color_map_grey.update(dict(zip(['TC1', 'TC2', 'TC3', 'TC4'], ['#d3d3d3']*8)))

In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'], 
                 color=['sample_origin'], 
               ncols=1, legend_loc='right margin', size = 1, title = '',
               frameon = False, return_fig=True, palette=['#808080','#ff7f0e'])


In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI, 
                 color=['cell_type_level2'], 
               ncols=1, legend_loc='right margin', size = 1, title = '',
               frameon = False, return_fig=True, palette=sample_color_map_grey)


In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'], 
                 color=['cell_type_level2'], 
               ncols=1, legend_loc='right margin', size = 1, title = '',
               frameon = False, return_fig=True, palette=sample_color_map_for_all)


In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI[(adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi') & 
                        (adata_all_noTC_cleanCI.obs['sample_origin'] == 'tumour')], 
                 color=['cell_type_level2'], 
               ncols=1, legend_loc='right margin', size = 1, title = '',
               frameon = False, return_fig=True, palette=sample_color_map_for_all)


In [ ]:
# double check CAFs
# CRC CAFA: MMP2, DCN, COL1A2
# CRC CAFB: ACTA2, PDGFA, TAGLN

fig = sc.pl.umap(adata_all_noTC_cleanCI, 
                 color=['EPCAM', 'CALD1', 'SMTN', 'PTPRC', 'PECAM1'],  # negetive marker: CALD1 is pericyte
               ncols=3, legend_loc='right margin', size = 1, 
               frameon = False, return_fig=True, palette=sample_color_map_for_all)


In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI, 
                 color=['MMP2', 'DCN', 'COL1A2','ACTA2', 'PDGFA', 'TAGLN',
                      'FAP','MFAP5', 'COL11A1', 'TNC', 'PDPN', 'ITGA11', 'CSPG4', # also CAF
                      'PDGFRA', 'PDGFRB', 'VIM'], # Fibroblast],  # negetive marker: CALD1 is pericyte
               ncols=3, legend_loc='right margin', size = 1, 
               frameon = False, return_fig=True, palette=sample_color_map_for_all)


### Other plots

In [ ]:
fig = sc.pl.umap(adata_all_noTC_cleanCI, 
                 color=['louvain'], 
               ncols=3, legend_loc='on data', size = 1, 
               frameon = False, return_fig=True)


In [ ]:
cluster_list = adata_epi_noTC_cleanCI.obs[['louvain', 'leiden', 'cell_identity', 
                                           'tumour_normal_normal', 'sample_origin',
                                           'Uhlitz_scANVI_noTC','iCMS_scANVI', 'inferCNV_result', 'sample_cap']]

In [ ]:
%%R

cluster_list %>%
filter(cell_identity == 'genomically_normal') %>%
group_by(sample_cap) %>%
mutate(n_sample = n()) %>%
group_by(sample_cap, louvain) %>%
mutate(n_louvain_per_sample = n(), frac_louvain = n_louvain_per_sample/n_sample) %>%
distinct(sample_cap, louvain, n_sample, n_louvain_per_sample, frac_louvain) %>%
select(sample_cap, louvain, n_sample, n_louvain_per_sample, frac_louvain) %>%
arrange(sample_cap, louvain) %>% 
filter(louvain == '9')

In [ ]:
%%R -i cluster_list -w 900 -h 900 -r 200

# proportion of each cell identity in each cluster

df = cluster_list %>%
group_by(louvain, cell_identity) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
distinct(louvain, cell_identity, sum, sum_louvain) %>%
arrange(louvain)


df$cell_identity = factor(df$cell_identity, levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

p = df %>%
ggplot(aes(x = sum, y = louvain, fill = cell_identity)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#ff7f0e',
                               '#1f77b4','#808080'),
                 labels = c('genomically\ntumour', 'genomically\nnormal', 'normal\nsample'),
                 name = 'Consensus call') +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
     axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = 'Louvain cluster\n') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                 ' (', rev(unique(df$sum_louvain)), ')'),
                expand = expansion(add = c(0, 1)))

print(p)

ggsave(file="louvain_cell_identity_bar.svg", width = 10, height = 10, units = "cm", dpi = 300)

In [ ]:
%%R -w 500 -h 900 -r 200

# proportion of each cell identity in each cluster

df = cluster_list %>%
group_by(louvain, tumour_normal_normal) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
distinct(louvain, tumour_normal_normal, sum, sum_louvain) %>%
arrange(louvain)


df$tumour_normal_normal = factor(df$tumour_normal_normal, levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'no confident assignment', 
                                                       'genomically_tumour')))


Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

p = df %>%
ggplot(aes(x = sum, y = louvain, fill = tumour_normal_normal)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity', show.legend=FALSE) +
scale_fill_manual(values = c('#ff7f0e', '#9b1ee3',
                               '#1f77b4','#808080'),
                  labels = c('genomically\ntumour', 'no confident\nassignment',
                             'genomically\nnormal', 'normal\nsample'),
                  name = 'Scitcem_Numbat') +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
     axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = '') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                              ' (', rev(unique(df$sum_louvain)), ')'),
                expand = expansion(add = c(0, 1)))

print(p)

ggsave(file="louvain_Scitcem_Numbat_bar.svg", width = 9, height = 10, units = "cm", dpi = 300)

In [ ]:
%%R -w 1200 -h 900 -r 200

# proportion of each cell identity in each cluster
suppressMessages(library(tidyverse))



df = cluster_list %>%
group_by(louvain, sample_origin) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
distinct(louvain, sample_origin, sum, sum_louvain) %>%
arrange(louvain) %>%
mutate(sample_origin = dplyr::recode(sample_origin,
                                     '\nnormal\nsample\n' = 'normal sample',
                                     'tumour\nsample' = 'tumour sample')) 

df$sample_origin = factor(df$sample_origin, levels = rev(c('normal sample','tumour sample')))


Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

p = df %>%
ggplot(aes(x = sum, y = louvain, fill = sample_origin)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#ff7f0e', '#808080')) +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
     axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = '') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                              ' (', rev(unique(df$sum_louvain)), ')'),
                expand = expansion(add = c(0, 1)))

print(p)

ggsave(file="louvain_sample_origin_bar.svg", width = 13, height = 10, units = "cm", dpi = 300)

'#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', '#8c564b',
       '#e377c2', '#b5bd61', '#17becf', '#aec7e8', '#ffbb78', '#98df8a',
       '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#dbdb8d', '#9edae5',
       '#ad494a', '#8c6d31'

In [ ]:
%%R -i cluster_list  -w 900 -h 700 -r 200

# proportion of cell type in each cluster

df = cluster_list %>%
group_by(louvain, sample_cap) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
distinct(louvain, sample_origin, sum, sum_louvain, sample_cap) %>%
arrange(louvain)


#Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(1,4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

df %>%
ggplot(aes(x = sum, y = louvain, fill = sample_cap)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', #'#8c564b',
       '#e377c2', #'#b5bd61', 
                             '#17becf', '#aec7e8', '#ffbb78', '#98df8a',
       '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#dbdb8d', '#9edae5',
       '#ad494a', '#8c6d31'),
                 name = 'sample') +
theme_void() + 
plot_theme_font_size(10) +
theme(axis.title.y = element_text(angle = 90),
     axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = '') +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                              ' (', rev(unique(df$sum_louvain)), ')')) +
scale_x_continuous(n.breaks = 3) +
guides(fill=guide_legend(ncol=2))

In [ ]:
%%R -i cluster_list,Uhlitz_scANVI_noTC_color_list  -w 900 -h 700 -r 200

# proportion of cell type in each cluster
suppressMessages(library(tidyverse))



df = cluster_list %>%
group_by(louvain, Uhlitz_scANVI_noTC) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
distinct(louvain, sample_origin, sum, sum_louvain, Uhlitz_scANVI_noTC) %>%
arrange(louvain)


Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

df %>%
ggplot(aes(x = sum, y = louvain, fill = Uhlitz_scANVI_noTC)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = Uhlitz_scANVI_noTC_color_list, name = 'Epithelial cell type') +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
     axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = 'Louvain cluster\n') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                              ' (', rev(unique(df$sum_louvain)), ')'))

In [ ]:
# the TC1 cells in cluster 7
adata_epi[(adata_epi.obs['louvain'] == '7') &(adata_epi.obs['Uhlitz_scANVI'] == 'TC1')].obs[['sample_cap']].value_counts().sort_index()

In [ ]:
%%R -i cluster_list,Uhlitz_scANVI_noTC_color_list  -w 1200 -h 700 -r 200

# proportion of cell type in each cluster

df = cluster_list %>%
group_by(louvain, Uhlitz_scANVI_noTC, sample_origin) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
group_by(sample_origin, louvain) %>%
mutate(sum_sample_louvain = n()) %>%
distinct(louvain, sample_origin, sum, sum_louvain, sum_sample_louvain, Uhlitz_scANVI_noTC) %>%
mutate(sample_origin = dplyr::recode(sample_origin,
                                     '\nnormal\nsample\n' = 'normal sample',
                                     'tumour\nsample' = 'tumour sample')) %>%
arrange(sample_origin,louvain)

Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(1,4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

df %>%
ggplot(aes(x = sum, y = louvain, fill = Uhlitz_scANVI_noTC)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = Uhlitz_scANVI_noTC_color_list) +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
      axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = '') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                 ' (', rev(c(unique(df$sum_sample_louvain)[1:19], '0')),'/',
                                 rev(unique(df$sum_sample_louvain)[20:39]), ')')) +
facet_grid(cols = vars(sample_origin),labeller = label_wrap_gen(width = 10)) +
guides(fill=guide_legend(title="Epithelial cell type"))


In [ ]:
%%R -i cluster_list,Uhlitz_scANVI_noTC_color_list_after  -w 1200 -h 700 -r 200

# proportion of cell type in each cluster
suppressMessages(library(tidyverse))



df = cluster_list %>%
filter(Uhlitz_scANVI_noTC != 'Tumour-like') %>%
mutate(louvain = droplevels(louvain),
       Uhlitz_scANVI_noTC = droplevels(Uhlitz_scANVI_noTC)) %>%
group_by(louvain, Uhlitz_scANVI_noTC, cell_identity) %>%
mutate(sum=n()) %>%
group_by(cell_identity, louvain) %>%
mutate(sum_sample_louvain = n()) %>%
mutate(cell_identity = factor(cell_identity, levels = c('normal_sample', 'genomically_normal'))) %>%
distinct(louvain, cell_identity, sum, sum_sample_louvain, Uhlitz_scANVI_noTC) %>%
arrange(cell_identity, louvain)

Tumour_island_color = rev(ifelse(seq(0,18,1) %in% c(1,4,5,12,13,15,16), '#ff7f0e', '#808080'))

df %>%
ggplot(aes(x = sum, y = louvain, fill = Uhlitz_scANVI_noTC)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = Uhlitz_scANVI_noTC_color_list_after) +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
      axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = 'Louvain cluster\n') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df[,c('louvain')])$louvain),
                                 ' (', rev(c(unique(df[,c('louvain','sum_sample_louvain')])$sum_sample_louvain[1:19])),'/',
                                 rev(c(unique(df[,c('louvain','sum_sample_louvain')])$sum_sample_louvain[20:34], '0',
                                       unique(df[,c('louvain','sum_sample_louvain')])$sum_sample_louvain[35:37])), ')')) +
facet_grid(cols = vars(cell_identity), 
          labeller = label_wrap_gen(width = 10))


In [ ]:
%%R -i cluster_list,Uhlitz_scANVI_noTC_color_list  -w 1200 -h 900 -r 200

# proportion of cell type in each cluster
suppressMessages(library(tidyverse))


df = cluster_list %>%
group_by(louvain, sample_cap, sample_origin) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
group_by(sample_cap, louvain) %>%
mutate(sum_sample_louvain = n()) %>%
distinct() %>%
arrange(louvain,sample_cap)

Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

df %>%
ggplot(aes(x = sum, y = louvain, fill = sample_cap)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
#scale_fill_manual(values = Uhlitz_scANVI_noTC_color_list) +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
      axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = 'Louvain cluster\n') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                 ' (', rev(c(unique(df$sum_sample_louvain)[1:19], '0')),'/',
                                 rev(unique(df$sum_sample_louvain)[20:39]), ')')) +
facet_grid(cols = vars(sample_origin)) +
scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', #'#8c564b',
       '#e377c2', #'#b5bd61', 
                             '#17becf', '#aec7e8', '#ffbb78', '#98df8a',
       '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#dbdb8d', '#9edae5',
       '#ad494a', '#8c6d31')) +
guides(fill=guide_legend(ncol=2))

In [ ]:
%%R -i cluster_list -w 500 -h 900 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))


df = cluster_list %>%
group_by(louvain, inferCNV_result) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
distinct(louvain, inferCNV_result, sum, sum_louvain) %>%
arrange(louvain)

Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

p = df %>%
ggplot(aes(x = sum, y = louvain, fill = inferCNV_result)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity', show.legend = FALSE) +
scale_fill_manual(values = c('#ff7f0e', '#1f77b4','#9b1ee3',
                               '#808080'),
                  labels = c('CNA', 'CNN',
                             'NA', 'normal\nsample'),
                  name = 'inferCNV') +

theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
     axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = '') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                              ' (', rev(unique(df$sum_louvain)), ')'),
                expand = expansion(add = c(0, 1)))
print(p)

ggsave(file="louvain_inferCNV_bar.svg", width = 9, height = 10, units = "cm", dpi = 300)


In [ ]:
%%R -i cluster_list -w 500 -h 900 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))



df = cluster_list %>%
group_by(louvain, iCMS_scANVI) %>%
mutate(sum=n()) %>%
group_by(louvain) %>%
mutate(sum_louvain = n()) %>%
distinct(louvain, iCMS_scANVI, sum, sum_louvain) %>%
arrange(louvain)

Tumour_island_color = rev(ifelse(seq(0,19,1) %in% c(4,5,12,13,15,16,19), '#ff7f0e', '#808080'))

p = df %>%
ggplot(aes(x = sum, y = louvain, fill = iCMS_scANVI)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity', show.legend = FALSE) +
scale_fill_manual(values = c('#FFB84C', '#E96479', '#1f77b4', '#808080')) +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90),
     axis.text.y = element_text(colour = Tumour_island_color)) +
labs(x = 'Proportion', 
     y = '') +
scale_x_continuous(n.breaks = 3) +
scale_y_discrete(limits = rev,
                 labels = paste0(rev(unique(df$louvain)),
                                              ' (', rev(unique(df$sum_louvain)), ')'),
                expand = expansion(add = c(0, 1)))

print(p)

ggsave(file="louvain_iCMS_bar.svg", width = 9, height = 10, units = "cm", dpi = 300)

### evaluation of clustering: ARI, NMI, homogeneity, completeness, silhouette


In [ ]:
ari_sample = adata_epi_noTC_cleanCI.obs['sample_origin'].copy()

In [ ]:
ari_ci = adata_epi_noTC_cleanCI.obs['cell_identity'].copy()
ari_ci = ari_ci.astype('str').replace('genomically_normal', 'normal_sample') # only 2 labels: GT and NN

In [ ]:
ari_icms = adata_epi_noTC_cleanCI.obs['iCMS_scANVI'].copy()
ari_icms = ari_icms.astype(str).replace('iCMS3\n(tumour sample)', 
                             'iCMS2\n(tumour sample)').replace('normal\n(tumour sample)',
                                                               'normal_sample')

In [ ]:
ari_infercnv = adata_epi_noTC_cleanCI.obs['inferCNV_result'].copy()
ari_infercnv = ari_infercnv.astype(str).replace('failed_sample', 
                             'CNA\n(tumour sample)').replace('CNN\n(tumour sample)',
                                                               'normal sample')

In [ ]:
ari_scitcem = adata_epi_noTC_cleanCI.obs['scitcem_call'].copy()
ari_scitcem = ari_scitcem.astype(str).replace('normal\n(tumour sample)', 
                             'normal_sample')

In [ ]:
ari_numbat = adata_epi_noTC_cleanCI.obs['numbat'].copy()
ari_numbat = ari_numbat.astype(str).replace('normal\n(tumour sample)', 
                             'normal_sample')

In [ ]:
ARI_res = [adjusted_rand_score(labels_true = adata_epi_noTC_cleanCI.obs['louvain'], 
                              labels_pred = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]

In [ ]:
ARI_res

In [ ]:
AMI_res = [adjusted_mutual_info_score(labels_true = adata_epi_noTC_cleanCI.obs['louvain'], 
                              labels_pred = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]

In [ ]:
AMI_res

In [ ]:
HCV_res = [homogeneity_completeness_v_measure(labels_true = adata_epi_noTC_cleanCI.obs['louvain'], 
                              labels_pred = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]
#V is identical to normalized_mutual_info_score with the 'arithmetic' option for averaging.

In [ ]:
S_pca_res = [silhouette_score(X = adata_epi_noTC_cleanCI.obsm['X_pca'], labels = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]

In [ ]:
S_pca_res

In [ ]:
S_umap_res = [silhouette_score(X = adata_epi_noTC_cleanCI.obsm['X_umap'], labels = i)
for i in [ari_sample, ari_infercnv, ari_icms, ari_scitcem, ari_numbat, ari_ci]]

In [ ]:
S_umap_res

In [ ]:
HCV_res_reshape = np.concatenate(HCV_res).reshape(6,3).transpose()

In [ ]:
#long format
cluster_eval_df = pd.DataFrame({'Label':['sample\norigin', 'inferCNV', 'iCMS','Scitcem', 'Numbat','consensus\ncall'],
             'Adjusted Rand Index': ARI_res,
             'Adjusted Mutual Information': AMI_res,
             'Homogeneity': HCV_res_reshape[0],
             'Completeness': HCV_res_reshape[1],
             'Silhouette (PCA)':S_pca_res,
             'Silhouette (UMAP)':S_umap_res}
             )

In [ ]:
cluster_eval_df_long = pd.melt(cluster_eval_df, 'Label', var_name='Metrics', value_name='Score')

In [ ]:
with plt.rc_context({'ytick.major.size':16, 'axes.titlesize':18}):
    plt.figure(figsize = (7,5))

    ax = sns.lineplot(data = cluster_eval_df_long[cluster_eval_df_long['Metrics'] == 'Homogeneity'], 
                      x = 'Label', y = 'Score', 
                      hue = 'Metrics', style = 'Metrics', 
                      markers=True, dashes=False, legend=None)
    #sns.move_legend(ax, "upper right", bbox_to_anchor=(1.5, 1.0), 
    #                frameon=False)

;

In [ ]:
with plt.rc_context({'ytick.major.size':16, 'axes.titlesize':18}):
    plt.figure(figsize = (7,5))

    sns.barplot(data = cluster_eval_df_long[cluster_eval_df_long['Metrics'] == 'Homogeneity'], 
                      x = 'Label', y = 'Score',
                palette = ['#f765b6','#f57379','#f5d107', '#228b22', '#47a2ed', '#008080'])
    plt.ylabel("Cluster homogeneity score")
    #sns.move_legend(ax, "upper right", bbox_to_anchor=(1.5, 1.0), 
    #                frameon=False)

;

### differences between normal vs tumour sample stem cells related louvain clusters

In [ ]:
# where are the normal stem cells?
adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['Uhlitz_scANVI'].isin(['Stem'])].obs[[ 'sample_origin','louvain',
                                                              'Uhlitz_scANVI_noTC']].value_counts().sort_index()

In [ ]:
adata_epi_noTC_cleanCI[adata_epi_noTC_cleanCI.obs['Uhlitz_scANVI'].isin(['Stem/TA'])].obs[[ 'sample_origin','louvain',
                                                              'Uhlitz_scANVI_noTC']].value_counts().sort_index()

### neighbor analysis

In [ ]:
# neighbors
adata_epi_noTC_cleanCI.obsp['connectivities'].toarray().shape

aa=scipy.sparse.csr_matrix((ad.obs[col].iloc[ad.obsp['distances'].nonzero()[1]]==category, 
                            ad.obsp['distances'].nonzero()))
aa.data[np.isnan(aa.data)]=0
aa.eliminate_zeros()
ad.obs['frac_neighbors']=aa.sum(1).A1/n_neighbors

In [ ]:
# for each row, grab the col number, 
# map to cell identity > box plot cell identity vs percentage of tumour neighbors  
neighbor_list = np.nonzero(adata_epi_noTC_cleanCI.obsp['connectivities'].toarray())

In [ ]:
neighbor_df = pd.DataFrame(np.vstack(neighbor_list).T, columns=['cell', 'neighbor'])

In [ ]:
neighbor_df['cell_identity'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['cell_identity'] 
                                for id in neighbor_df['cell']]

In [ ]:
neighbor_df['sample_origin'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['sample_origin'] 
                                for id in neighbor_df['cell']]

In [ ]:
neighbor_df['patient'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['patient'] 
                          for id in neighbor_df['cell']]

In [ ]:
neighbor_df['sample'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['sample_cap'] 
                         for id in neighbor_df['cell']]

In [ ]:
neighbor_df['neighbor_identity'] = [1 
                                    if adata_epi_noTC_cleanCI.obs.iloc[id]['cell_identity'] == 'genomically_tumour' else 0
                                    for id in neighbor_df['neighbor']]

In [ ]:
neighbor_df['neighbor_identity_2'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['cell_identity'] 
                                      for id in neighbor_df['neighbor']]

In [ ]:
neighbor_df['neighbor_sample_origin'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['sample_origin'] 
                                         for id in neighbor_df['neighbor']]

In [ ]:
neighbor_df['neighbor_patient'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['patient'] 
                                   for id in neighbor_df['neighbor']]

In [ ]:
neighbor_df['neighbor_sample'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['sample_cap'] 
                                  for id in neighbor_df['neighbor']]

In [ ]:
neighbor_df['louvain'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['louvain'] 
                                   for id in neighbor_df['cell']]

In [ ]:
neighbor_df['neighbor_louvain'] = [adata_epi_noTC_cleanCI.obs.iloc[id]['louvain'] 
                                   for id in neighbor_df['neighbor']]

In [ ]:
neighbor_df_tumour_percentage = neighbor_df.groupby(['cell'])['neighbor_identity'].sum()/neighbor_df['cell'].value_counts().sort_index()

In [ ]:
neighbor_df_tumour_percentage = neighbor_df_tumour_percentage.to_frame('percent').reset_index(drop=True)

In [ ]:
neighbor_df_tumour_percentage = neighbor_df_tumour_percentage.merge(pd.DataFrame(adata_epi_noTC_cleanCI.obs['cell_identity'].reset_index()),
                                   left_index=True, right_index=True)

In [ ]:
neighbor_df_tumour_percentage['percent'].hist()

In [ ]:
neighbor_df_tumour_percentage = neighbor_df_tumour_percentage[(neighbor_df_tumour_percentage['cell_identity'] != 'no confident assignment')]

neighbor_df_tumour_percentage['cell_identity'] = neighbor_df_tumour_percentage['cell_identity'].cat.remove_unused_categories()


In [ ]:
ax = sns.boxplot(data = neighbor_df_tumour_percentage, 
                 x = 'cell_identity', y = 'percent', width = 0.4,
            palette = ['#ff7f0e', #'#9b1ee3',
                       '#1f77b4','#808080'], linewidth=1,
                 flierprops={"marker": "o"}, fliersize= 2)
#sns.stripplot(data = neighbor_df_tumour_percentage, x = 'cell_identity', y = 'percent',
#            palette = ['#ff7f0e', '#9b1ee3','#1f77b4','#808080'], ax = ax, size=2)
plt.xticks(rotation=90)
plt.ylabel('Percentage of genomically tumour cells\namong neighbors\n')
plt.xlabel('\nCell identity')
;


In [ ]:
%%R -i neighbor_df -w 1200 -h 600 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
group_by(cell_identity, neighbor_sample) %>%
mutate(sum=n()) %>%
group_by(cell_identity) %>%
mutate(sum_cell= n()) %>%
distinct(cell_identity, neighbor_sample, sum, sum_cell) %>%
arrange(cell_identity)

df$cell_identity = factor(df$cell_identity,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df %>%
ggplot(aes(x = sum, y = cell_identity, fill = neighbor_sample)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', #'#8c564b',
       '#e377c2', #'#b5bd61', 
                             '#17becf', '#aec7e8', '#ffbb78', '#98df8a',
       '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#dbdb8d', '#9edae5',
       '#ad494a', '#8c6d31')) +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90)) +
#labs(x = 'Proportion', 
#     y = 'Louvain cluster\n') +
#scale_x_discrete(limits = rev) +
guides(fill=guide_legend(ncol=2))

In [ ]:
%%R -i neighbor_df -w 1200 -h 400 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
group_by(cell_identity, neighbor_sample_origin) %>%
mutate(sum=n()) %>%
group_by(cell_identity) %>%
mutate(sum_cell= n()) %>%
distinct(cell_identity, neighbor_sample_origin, sum, sum_cell) %>%
mutate(neighbor_sample_origin = dplyr::recode(neighbor_sample_origin,
                                     '\nnormal\nsample\n' = 'normal sample',
                                     'tumour\nsample' = 'tumour sample')) %>%
arrange(cell_identity) 

df$cell_identity = factor(df$cell_identity,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df$neighbor_sample_origin = factor(df$neighbor_sample_origin,levels = rev(c('normal sample',
                                                       'tumour sample')))

df %>%
ggplot(aes(x = sum, y = cell_identity, fill = neighbor_sample_origin)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#ff7f0e', '#808080')) +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Cell identity\n') #+
#scale_x_discrete(limits = rev) +
#guides(fill=guide_legend(ncol=2))

In [ ]:
%%R -i neighbor_df -w 1200 -h 800 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
group_by(sample, neighbor_sample) %>%
mutate(sum=n()) %>%
group_by(sample) %>%
mutate(sum_cell= n()) %>%
distinct(sample, neighbor_sample, sum, sum_cell) %>%
arrange(sample)

df %>%
ggplot(aes(x = sum, y = sample, fill = neighbor_sample)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', #'#8c564b',
       '#e377c2', #'#b5bd61', 
                             '#17becf', '#aec7e8', '#ffbb78', '#98df8a',
       '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#dbdb8d', '#9edae5',
       '#ad494a', '#8c6d31')) +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90)) +
#labs(x = 'Proportion', 
#     y = 'Louvain cluster\n') +
scale_y_discrete(limits = rev) +
guides(fill=guide_legend(ncol=2))

In [ ]:
louvain_colors = adata_epi.uns['louvain_colors']

In [ ]:
%%R -i neighbor_df,louvain_colors -w 1200 -h 800 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
group_by(sample, louvain) %>%
mutate(sum=n()) %>%
group_by(sample) %>%
mutate(sum_cell= n()) %>%
distinct(sample, louvain, sum, sum_cell) %>%
arrange(sample)

df %>%
ggplot(aes(x = sum, y = sample, fill = louvain)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_y_discrete(limits = rev) +
guides(fill=guide_legend(ncol=2)) +
scale_fill_manual(values = louvain_colors)

In [ ]:
%%R -i neighbor_df -w 1200 -h 300 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
filter(louvain == '9') %>%
group_by(sample) %>%
mutate(sum=n()) %>%
distinct(sample, louvain, sum) %>%
arrange(sample)

df %>%
ggplot(aes(x = sum, y = louvain, fill = sample)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Louvain\n') +
scale_y_discrete(limits = rev) +
guides(fill=guide_legend(ncol=4)) +
scale_fill_manual(values = c('#ff7f0e', '#279e68', '#aa40fc', 
       '#e377c2', '#aec7e8', '#98df8a',
       '#ff9896', '#c5b0d5', '#f7b6d2', '#9edae5',
       '#ad494a', '#8c6d31')) 

- for each patient, get each cell's fraction of neighbors from the same patient, separately for genomically normal / genomically tumor / normal sample cells, and normalize that fraction by the fraction of cells of that patient in the entire dataset
- for each patient, what's the fraction of their genomically normal cells that end up in cluster 9?

In [ ]:
%%R

# for each patient, if any sample is over-represented in the neighbors 
# vs in the whole adata, fraction of each patient

pt_frac = neighbor_df %>%
select(cell, patient, cell_identity) %>%
distinct() %>%
mutate(total = n()) %>%
group_by(patient) %>%
mutate(n_cell = n(), frac = n_cell/total) %>%
group_by(patient, cell_identity) %>%
mutate(n_CI_per_pt = n(), 
       CI_frac = n_CI_per_pt/n_cell, 
       CI_frac_global = n_CI_per_pt/total) %>%
distinct(patient, n_cell, n_CI_per_pt, frac, CI_frac, CI_frac_global) %>%
arrange(patient)

pt_frac

In [ ]:
%%R  

# for each patient each CI, how many neighbors are from the same patient?

df = neighbor_df %>%
group_by(patient, cell_identity, neighbor_patient) %>% 
mutate(n_Npt = n()) %>%
mutate(n_Npt_per_NI = n(), 
       frac_Npt_NI = n_Npt_per_NI/n_NI) %>% # divide 2 by 1 = fraction of samples in each CI
distinct(patient, cell_identity, neighbor_patient, neighbor_identity_2, n_NI, n_Npt_per_NI, frac_Npt_NI) %>%
arrange(patient, cell_identity, neighbor_patient, neighbor_identity_2) %>%
select(patient, cell_identity, neighbor_patient, neighbor_identity_2, n_NI, n_Npt_per_NI, frac_Npt_NI)

df$neighbor_identity_2 = factor(df$neighbor_identity_2, levels = c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour'))

df

In [ ]:
%%R 
df %>%
ggplot() +
geom_histogram(aes(frac_Npt_NI))

In [ ]:
%%R -w 2000 -h 1000 -r 200

df %>% left_join(sample_frac, by = 'patient') %>%
mutate(over_rep = frac_Npt_NI/frac) %>%
select(patient, neighbor_patient, cell_identity, neighbor_identity_2, over_rep) %>% 
ggplot() +
geom_boxplot(aes(x = patient, y = over_rep, fill = neighbor_identity_2), width = 0.8) +
geom_hline(yintercept = 1, color = 'red') +
facet_grid(rows = vars(cell_identity)) +
theme_void() + 
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90)) +
scale_y_continuous(n.breaks = 3) +
labs(y = 'fraction of sample')


In [ ]:
%%R -i neighbor_df -w 1200 -h 300 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
filter(louvain == '9') %>%
group_by(cell_identity, louvain) %>%
mutate(sum=n()) %>%
group_by(cell_identity) %>%
mutate(sum_cell= n()) %>%
distinct(cell_identity, louvain, sum, sum_cell) %>%
arrange(cell_identity)

df$cell_identity = factor(df$cell_identity,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df %>%
ggplot(aes(x = sum, y = louvain, fill = cell_identity)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Louvain\n') +
scale_y_discrete(limits = rev) +
guides(fill=guide_legend(ncol=1)) +
scale_fill_manual(values = c('#ff7f0e', 
                             '#1f77b4','#808080')) 

In [ ]:
%%R -i neighbor_df -w 1200 -h 300 -r 200

# proportion of inferCNV in each cluster
suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
filter(louvain == '9') %>%
group_by(neighbor_identity_2, louvain) %>%
mutate(sum=n()) %>%
group_by(neighbor_identity_2) %>%
mutate(sum_cell= n()) %>%
distinct(neighbor_identity_2, louvain, sum, sum_cell) %>%
arrange(neighbor_identity_2)

df$neighbor_identity_2 = factor(df$neighbor_identity_2,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df %>%
ggplot(aes(x = sum, y = louvain, fill = neighbor_identity_2)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Louvain\n') +
scale_y_discrete(limits = rev) +
guides(fill=guide_legend(ncol=1)) +
scale_fill_manual(values = c('#ff7f0e', 
                             '#1f77b4','#808080')) 

In [ ]:
%%R -i neighbor_df -w 1200 -h 400 -r 200

suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
filter(louvain == '9') %>%
group_by(neighbor_sample, louvain) %>%
mutate(sum=n()) %>%
group_by(neighbor_sample) %>%
mutate(sum_cell= n()) %>%
distinct(neighbor_sample, louvain, sum, sum_cell) %>%
arrange(neighbor_sample)

df %>%
ggplot(aes(x = sum, y = louvain, fill = neighbor_sample)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Louvain\n') +
scale_y_discrete(limits = rev) +
guides(fill=guide_legend(ncol=5)) +
scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', #'#8c564b',
       '#e377c2', #'#b5bd61', 
                             '#17becf', '#aec7e8', '#ffbb78', '#98df8a',
       '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#dbdb8d', '#9edae5',
       '#ad494a', '#8c6d31')) 

In [ ]:
%%R -i neighbor_df -w 2000 -h 800 -r 200

suppressMessages(library(tidyverse))

plot_theme =  theme(title = element_text(size = 12),
                   axis.title = element_text(size = 12),
                   axis.text = element_text(size = 10),
                   legend.title =  element_text(size = 8),
                   text = element_text(size = 8),
                   plot.margin = margin(0,0,0,0,"cm")) 

df = neighbor_df %>%
group_by(sample, neighbor_sample, cell_identity) %>%
mutate(sum=n()) %>%
group_by(sample) %>%
mutate(sum_cell= n()) %>%
distinct(sample, neighbor_sample, sum, sum_cell, cell_identity) %>%
arrange(sample)

df$cell_identity = factor(df$cell_identity,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df %>%
ggplot(aes(x = sum, y = sample, fill = neighbor_sample)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', #'#8c564b',
       '#e377c2', #'#b5bd61', 
                             '#17becf', '#aec7e8', '#ffbb78', '#98df8a',
       '#ff9896', '#c5b0d5', '#c49c94', '#f7b6d2', '#dbdb8d', '#9edae5',
       '#ad494a', '#8c6d31')) +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90)#,
      #axis.text.x = element_text(angle = 90)
     ) +
labs(x = 'Proportion', 
     y = 'Sample\n') +
scale_y_discrete(limits = rev) +
guides(fill=guide_legend(ncol=2)) + 
facet_grid(cols = vars(cell_identity))

In [ ]:
adata_epi.uns['patient_colors']

In [ ]:
%%R -i neighbor_df -w 2000 -h 800 -r 200

df = neighbor_df %>%
group_by(patient, neighbor_patient, cell_identity) %>%
mutate(sum=n()) %>%
distinct(patient, neighbor_patient, sum, cell_identity) %>%
arrange(patient)

df$cell_identity = factor(df$cell_identity,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df %>%
ggplot(aes(x = sum, y = patient, fill = neighbor_patient)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90)#,
      #axis.text.x = element_text(angle = 90)
     ) +
labs(x = 'Proportion', 
     y = 'Patient\n') +
scale_y_discrete(limits = rev) +
guides(fill=guide_legend(ncol=2)) + 
facet_grid(cols = vars(cell_identity)) +
scale_x_continuous(breaks = 3) +
scale_fill_manual(values = c('#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', '#8c564b',
       '#e377c2', '#b5bd61', '#17becf', '#aec7e8'))

In [ ]:
%%R -i neighbor_df -w 2000 -h 800 -r 200

df %>%
mutate(same_pt = ifelse(patient == neighbor_patient, 'Yes', 'no')) %>%
ggplot(aes(x = sum, y = patient, fill = same_pt)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
theme_void() + 
plot_theme_font_size(12) +
theme(axis.title.y = element_text(angle = 90)#,
      #axis.text.x = element_text(angle = 90)
     ) +
labs(x = 'Proportion', 
     y = 'Patient\n') +
scale_y_discrete(limits = rev) +
facet_grid(cols = vars(cell_identity)) +
scale_x_continuous(breaks = 3) +
scale_fill_manual(values = c('grey', 'steelblue'),
                  name = 'From the same patient')

In [ ]:
%%R 

df = neighbor_df %>%
mutate(same_pt = ifelse(patient == neighbor_patient, 1, 0)) %>%
group_by(cell) %>%
mutate(n_Npt = n(), Nfrac = sum(same_pt)/n_Npt) %>%
distinct(patient, cell, cell_identity, Nfrac) %>%
arrange(patient, cell)

df$cell_identity = factor(df$cell_identity,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

print(df)

In [ ]:
%%R -w 800 -h 1000 -r 200

df %>%
ggplot() +
geom_boxplot(aes(x = patient, y = Nfrac), outlier.size = 0.5) +
facet_grid(rows = vars(cell_identity)) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90)) +
labs(y = 'Fraction of neighbor from the same patient\n')

In [ ]:
%%R

# divid by patient frac
df = df %>%
left_join(pt_frac, by = c('patient', 'cell_identity')) %>%
mutate(over_rep = Nfrac/frac, over_rep_CI = Nfrac/CI_frac, over_rep_CI_global = Nfrac/CI_frac_global) %>%
arrange(cell, cell_identity)

df$cell_identity = factor(df$cell_identity,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df

In [ ]:
%%R -w 500 -h 1500 -r 200


df %>%
ggplot() +
geom_boxplot(aes(x = cell_identity, y = over_rep), outlier.size = 0.5) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90),
     axis.ticks.y = element_line('black', linewidth = 1),
      axis.ticks.length=unit(0.1, "cm"),
      axis.text.y = element_text(margin = margin(t = 0, r = 7, b = 0, l = 0)),
      axis.line = element_line()) +
labs(y = 'fractions of same-patient neighbor\nnormalised by\nfraction of patient in dataset\n then averaged by sample\n') +
scale_x_discrete(labels = c('GT', 'GN', 'NS')) +
facet_grid(rows = vars(patient))

In [ ]:
%%R -w 500 -h 500 -r 200


df %>%
group_by(patient) %>%
mutate(mean_over_rep = mean(over_rep)) %>%
ggplot() +
geom_boxplot(aes(x = cell_identity, y = mean_over_rep), outlier.size = 0.5) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90),
     axis.ticks.y = element_line('black', linewidth = 1),
      axis.ticks.length=unit(0.1, "cm"),
      axis.text.y = element_text(margin = margin(t = 0, r = 7, b = 0, l = 0)),
      axis.line = element_line()) +
labs(y = 'fractions of same-patient neighbor\nnormalised by\nfraction of patient in dataset\n then averaged by sample\n') +
scale_x_discrete(labels = c('GT', 'GN', 'NS'))


In [ ]:
%%R -w 500 -h 500 -r 200


df %>%
ggplot() +
geom_boxplot(aes(x = cell_identity, y = over_rep), outlier.size = 0.5) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90),
     axis.ticks.y = element_line('black', linewidth = 1),
      axis.ticks.length=unit(0.1, "cm"),
      axis.text.y = element_text(margin = margin(t = 0, r = 7, b = 0, l = 0)),
      axis.line = element_line()) +
labs(y = 'fractions of same-patient neighbor\nnormalised by\nfraction of patient in dataset\n') +
scale_x_discrete(labels = c('GT', 'GN', 'NS'))


In [ ]:
%%R -w 800 -h 1000 -r 200

# divid by patient frac
df %>%
ggplot() +
geom_boxplot(aes(x = patient, y = over_rep_CI), outlier.size = 0.5) +
facet_grid(rows = vars(cell_identity)) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90)) +
labs(y = 'fration of same-patient neighbor normalised\nby fraction of patient in dataset\n')


In [ ]:
%%R -w 500 -h 500 -r 200

# divid by patient frac
df %>%
ggplot() +
geom_boxplot(aes(x = cell_identity, y = over_rep_CI), outlier.size = 0.5) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90)) +
labs(y = 'fration of same-patient neighbor normalised\nby fraction of patient in dataset\n') +
scale_y_continuous(trans='log10') +
scale_x_discrete(labels = c('GT', 'GN', 'NS'))

In [ ]:
%%R -w 800 -h 1000 -r 200

# divid by patient frac
df %>%
ggplot() +
geom_boxplot(aes(x = patient, y = over_rep_CI_global), outlier.size = 0.5) +
facet_grid(rows = vars(cell_identity)) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90)) +
labs(y = 'fration of same-patient neighbor normalised\nby fraction of patient in dataset\n') +
scale_x_discrete(labels = c('GT', 'GN', 'NS'))


In [ ]:
%%R -w 500 -h 500 -r 200

# divid by patient frac
df %>%
ggplot() +
geom_boxplot(aes(x = cell_identity, y = over_rep_CI_global), outlier.size = 0.5) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90)) +
labs(y = 'fration of same-patient neighbor normalised\nby fraction of patient in dataset\n') +
scale_y_continuous(trans='log10') +
scale_x_discrete(labels = c('GT', 'GN', 'NS'))

In [ ]:
%%R -w 500 -h 500 -r 200


df %>%
group_by(patient) %>%
mutate(mean_over_rep_CI_global = mean(over_rep_CI_global)) %>%
ggplot() +
geom_boxplot(aes(x = cell_identity, y = mean_over_rep_CI_global), outlier.size = 0.5) +
theme_void() +
plot_theme_font_size(8) +
theme(axis.title.y = element_text(angle = 90),
     axis.ticks.y = element_line('black', linewidth = 1),
      axis.ticks.length=unit(0.1, "cm"),
      axis.text.y = element_text(margin = margin(t = 0, r = 7, b = 0, l = 0)),
      axis.line = element_line()) +
labs(y = 'fractions of same-patient neighbor\nnormalised by\nfraction of per call per patient in dataset\n then averaged by sample\n') +
scale_x_discrete(labels = c('GT', 'GN', 'NS'))


In [ ]:
%%R -i neighbor_df -w 1200 -h 400 -r 200

# proportion of inferCNV in each cluster
df = neighbor_df %>%
group_by(cell_identity, neighbor_identity_2) %>%
mutate(sum=n()) %>%
group_by(cell_identity) %>%
mutate(sum_cell= n()) %>%
distinct(cell_identity, neighbor_identity_2, sum, sum_cell) %>%
arrange(cell_identity) 

df$cell_identity = factor(df$cell_identity,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df$neighbor_identity_2 = factor(df$neighbor_identity_2,levels = rev(c('normal_sample',
                                                       'genomically_normal',
                                                       'genomically_tumour')))

df %>%
ggplot(aes(x = sum, y = cell_identity, fill = neighbor_identity_2)) +
geom_col(width = 0.8, position = 'fill', stat = 'identity') +
scale_fill_manual(values = c('#ff7f0e', #'#9b1ee3',
                             '#1f77b4','#808080'),
                 ) +
theme_void() + 
plot_theme +
theme(axis.title.y = element_text(angle = 90),
      axis.text.x = element_text(angle = 90)) +
labs(x = 'Proportion', 
     y = 'Cell identity\n') #+
#scale_x_discrete(limits = rev) +
#guides(fill=guide_legend(ncol=2))

In [ ]:
%%R

saveRDS(neighbor_df, file = "202401_neighbor_df.rds")